<a href="https://colab.research.google.com/github/monacofj/sourcetide/blob/main/sourcetide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SourceTide - Analytics for collaborative development dynamics**



---
*Copyright (c) 2025, Monaco F. J. <monaco@usp.br> </br>
This is free software distributed under the GNU General Public License vr. 3.0.*

*Open Source/Science, CCOS-ICMC - University of São Paulo.*










# USER SETUP

## Repository

In [9]:
## Identify GitHub repository and owner.

owner = "fossguild"  # <--- Replace with the actual owner username
repository_name = "naja"  # <--- Replace with the actual repository name

# GitHub Token
#
# To access the GitHub API, you'll need a personal access token. You can
# generate one in your GitHub account settings under "Developer settings" >
# "Personal  access tokens". Grant the token the necessary permissions
# (e.g., `repo` scope for full control or more specific scopes like
# `public_repo` for public repositories). In Colab, you can store your token
# securely in the secrets manager under the key icon in the left panel.
# Add a new secret with the name `GITHUB_TOKEN` and paste your token as
# the value.

## Period

In [10]:
## Define analysis period in weeks.

num_weeks = 4

## Report

In [11]:
## Configure report generation settings.

# Should an HTML report file be generated?
# Answer with 'y', 'Y' (for Yes) or 'n', 'N' (for No)

generate_report_value = 'y'

# Should the analysis date be appended to the report folder name?
# Answer with 'y', 'Y' (for Yes) or 'n', 'N' (for No)
append_date_to_folder_name = 'y'

# Base path where the report folder will be saved (Google Drive)
report_path = 'My Drive/'

In [12]:
## Display a summary of the GitHub repository.

try:
    repo = g.get_user(owner).get_repo(repository_name)
    print(f"Repository Name: {repo.name}")
    print(f"Repository Description: {repo.description}")
    print(f"Repository URL: {repo.html_url}")
    print(f"Stars: {repo.stargazers_count}")
    print(f"Forks: {repo.forks_count}")
    print(f"Created At: {repo.created_at}")
    print(f"Last Updated At: {repo.updated_at}")

    # The analysis_run_date is now defined in the 'Analysis Run Date' cell in USER SETUP.
    # This cell will only print it.
    print(f"Analysis Run Date: {analysis_run_date}")

except Exception as e:
    print(f"Error fetching repository information: {e}")
    # If you see an error here AFTER replacing the owner and repository_name,
    # please copy and paste the full error message so I can assist you.

Error fetching repository information: name 'g' is not defined


## Análise

In [13]:
## Compute the date of the analysis execution.

from datetime import datetime

# Define the single source of truth for the analysis run date
analysis_run_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f"Analysis started at: {analysis_run_date}")

Analysis started at: 2025-11-30 14:06:28


## Score

In [14]:
## Configure score parameters.

## Chosse the parameters to be used to compute the contribution score
##
## activity = A * (REMI>0)   +   (1-A) (RDMI>0)
## reward   = R *  REMI      +   (1-R)  RDMI
## score    = S * activity   +   (1-S)  reward
##
## Interpretation:
##
## * activity a binary weighted measure of minimal participation in both
##            discussions and coding. The parameter A says how much each
##            dimension is important: A=1 means only discussion counts;
##            A=0 means only coding counts; A=0.5 both count equally.
##            Activity varies from 0 to 1.
##
##  * reward  is a weighted measure of participation in both discussion
##            and coding. The parameter R says how much each dimension is
##            important: R=0 means only discussion counts; R=1 means only
##            coding counts; R=0.5 both count equally.
##            Reward valies from 0 to 1.
##
##  * score   is a weighted combination of activity and reward. The parameter
##            S says which is more imporant: S=1 means that we're exclusively
##            interested in the fact that there was any participation, and
##            not in how intensive this participation was. S=0 means that the
##            score is proportional to the participation intensity. S=0.5
##            mean that half of the score is based on whether was any
##            participation, and the other half is a bonification for the
##            intensity of the work done.

A_factor = 0.5
R_factor = 0.5
S_factor = 0.5


# CONFIGURE TOOLS

In [15]:
## Mount GDrive and create (if needed) the report folder.

from google.colab import drive
import os
import ipywidgets as widgets # Import ipywidgets to access the selector
from IPython.display import display, Markdown
from datetime import datetime

# Mount Google Drive
drive.mount('/content/gdrive')

# Define the report_folder_name using repository_name and analysis_run_date
# repository_name is from the 'which-repo' cell (borUqKOx6vVf)
# analysis_run_date is from the 'Get repository info' cell (-zplV57Ea4LY)

# Ensure analysis_run_date is available (defined in -zplV57Ea4LY)
if 'analysis_run_date' not in globals():
    # Fallback if analysis_run_date hasn't been set yet
    current_date_str = datetime.now().strftime('%Y-%m-%d')
else:
    # Use the date from the analysis, formatted for folder names
    # analysis_run_date is already a string '%Y-%m-%d %H:%M:%S', reformat for folder name
    current_date_obj = datetime.strptime(analysis_run_date, '%Y-%m-%d %H:%M:%S')
    current_date_str = current_date_obj.strftime('%Y-%m-%d')

# Construct report_folder_name based on append_date_to_folder_name
if 'append_date_to_folder_name' in globals() and append_date_to_folder_name.lower() == 'y':
    report_folder_name = f"{repository_name}-{current_date_str}"
else:
    report_folder_name = repository_name

# Define the output folder path using report_path (from 'report' cell) and report_folder_name
# Ensure report_path is defined in the 'report' cell (OnyMCQQSVi00)
output_folder = os.path.join('/content/gdrive', report_path, report_folder_name)

# Create the folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
print(f"Output folder '{output_folder}' ensured to exist.")

Mounted at /content/gdrive
Output folder '/content/gdrive/My Drive/naja-2025-11-30' ensured to exist.


In [16]:
## Install package for the GitHub API

!pip --quiet install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.2 MB/s eta 0:00:00


In [17]:
## Define a function to save Plotly graphics.

import plotly.io as pio
import os
from IPython.display import display, Markdown

def save_plotly_figure_as_html(fig, filename):
    """
    Saves a Plotly figure as an interactive HTML file in the designated output folder.

    Args:
        fig (plotly.graph_objects.Figure): The Plotly figure object to save.
        filename (str): The desired filename (e.g., 'my_plot.html').
    """
    global output_folder

    # Ensure filename has .html extension
    if not filename.lower().endswith('.html'):
        filename = os.path.splitext(filename)[0] + '.html'

    full_path = os.path.join(output_folder, filename)
    pio.write_html(fig, full_path, auto_open=False) # auto_open=False to prevent browser from opening
    print(f"Figure saved to: {full_path}")


def save_fig(fig, filename):
    """
    Conditionally saves a Plotly figure as an interactive HTML file
    if report generation is enabled via the 'generate_report_value' variable.

    Args:
        fig (plotly.graph_objects.Figure): The Plotly figure object to save.
        filename (str): The desired filename (e.g., 'my_plot.html').
    """
    # Access the global variable 'generate_report_value' directly
    if 'generate_report_value' in globals() and generate_report_value.lower() == 'y':
        save_plotly_figure_as_html(fig, filename)
    else:
        print(f"Report generation is not enabled. Skipping saving '{filename}'.")

print("Function 'save_fig' defined.")

Function 'save_fig' defined.


In [18]:
## Compute the analysis timedelta.

from datetime import timedelta
from IPython.display import display, Markdown

# Calculate the corresponding timedelta from num_weeks (defined in the 'period' cell)
selected_timedelta = timedelta(weeks=num_weeks)

# Create a string representation for the report
selected_period_str = f'{num_weeks} week' if num_weeks == 1 else f'{num_weeks} weeks'

# Display the selected period for confirmation
display(Markdown(f'**Analysis Period: {selected_period_str}**'))

**Analysis Period: 4 weeks**

In [19]:
## Authenticate in GitHub.

import os
from google.colab import userdata
from github import Github

# Retrieve the GitHub token from Colab secrets
github_token = userdata.get('GITHUB_TOKEN')

if github_token is None:
    print("GitHub token not found. Please add it to Colab secrets with the name 'GITHUB_TOKEN'.")
else:
    # Authenticate with the GitHub API
    g = Github(github_token)
    print("Successfully authenticated with GitHub API.")

Successfully authenticated with GitHub API.


/tmp/ipython-input-429619539.py:14: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(github_token)


# FETCH DEVELOPERS

## Get repository info

In [20]:
## Get and display repository information.

try:
    repo = g.get_user(owner).get_repo(repository_name)
    print(f"Repository Name: {repo.name}")
    print(f"Repository Description: {repo.description}")
    print(f"Repository URL: {repo.html_url}")
    print(f"Stars: {repo.stargazers_count}")
    print(f"Forks: {repo.forks_count}")
    print(f"Created At: {repo.created_at}")
    print(f"Last Updated At: {repo.updated_at}")

    # Capture and print the current analysis run date
    from datetime import datetime # Ensure datetime is imported
    analysis_run_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"Analysis Run Date: {analysis_run_date}")

except Exception as e:
    print(f"Error fetching repository information: {e}")
    # If you see an error here AFTER replacing the owner and repository_name,
    # please copy and paste the full error message so I can assist you.

Repository Name: naja
Repository Description: Naja is an implementation of the Snake Game based on KobraPy programming exercise (2025)
Repository URL: https://github.com/fossguild/naja
Stars: 17
Forks: 39
Created At: 2025-10-01 18:48:04+00:00
Last Updated At: 2025-11-29 16:52:34+00:00
Analysis Run Date: 2025-11-30 14:06:54


## Get contributors
All people that contributed to the project, even if not with repository access (i.e. including external contributors).

In [21]:
## Fetch and filter non-bot contributors.

raw_contributors = repo.get_contributors() # Fetch all contributors first

filtered_contributors = []
bot_logins_to_exclude = ['Copilot'] # Explicitly exclude Copilot as it caused 404 in previous cells

for contributor in raw_contributors:
    # Check if contributor is a bot based on type or common naming conventions
    is_bot = (
        contributor.type == 'Bot' or
        contributor.login.endswith('[bot]') or
        contributor.login in bot_logins_to_exclude
    )

    if not is_bot:
        filtered_contributors.append(contributor)

all_contributors = filtered_contributors # Update all_contributors with the filtered list

print(f"Found {len(all_contributors)} non-bot contributors.")

# Display the login, name, and number of contributions for the first few non-bot contributors
print("\nFirst 5 non-bot contributors:")
for i, contributor in enumerate(all_contributors):
    if i >= 5:
        break
    # Get contributor name, handling cases where it might be None
    contributor_name = contributor.name if contributor.name else "N/A"
    # Use f-string formatting to align output
    print(f"- {contributor.login:<20}  {contributor_name:<30}    Contributions: {contributor.contributions}")

Found 40 non-bot contributors.

First 5 non-bot contributors:
- lfelipediniz          Luiz Felipe                       Contributions: 42
- LeticiaBN             Leticia Barbosa Neves             Contributions: 16
- caue-paiva            Caue Paiva                        Contributions: 9
- monacofj              Monaco F. J.                      Contributions: 7
- bdogabriel            Gabriel Barbosa                   Contributions: 6


## Get collaborators
This is a list of everyone with access to the repostory, even those who never effectively contributed.

In [22]:
## Fetch repository collaborators.

collaborators = repo.get_collaborators()

print(f"Found {collaborators.totalCount} collaborators with access to the repository.")

print("\nFirst 5 collaborators:")
for i, collaborator in enumerate(collaborators):
    if i >= 5:
        break
    # Get collaborator name, handling cases where it might be None
    collaborator_name = collaborator.name if collaborator.name else "N/A"

    permissions_list = []
    if collaborator.permissions.admin: permissions_list.append('admin')
    if collaborator.permissions.push: permissions_list.append('push')
    if collaborator.permissions.pull: permissions_list.append('pull')

    print(f"- {collaborator.login:<20}  Name: {collaborator_name:<30}  Permissions: {', '.join(permissions_list) if permissions_list else 'None'}")

Found 65 collaborators with access to the repository.

First 5 collaborators:
- monacofj              Name: Monaco F. J.                    Permissions: admin, push, pull
- LiviaLelis            Name: Lívia Lelis                     Permissions: push, pull
- trqt                  Name: Guilherme Torquato              Permissions: push, pull
- jotaefepinho          Name: João Francisco Pinho            Permissions: push, pull
- chonkachu             Name: N/A                             Permissions: push, pull


## Consolidate developers
Create a dataframe with all collaborators (people with repository access), plus anyone else that has contributed (external contributors).

In [25]:
## Consolidate unique developers into a DataFrame.

import pandas as pd

# --- 1. Prepare base information for all unique individuals (collaborators + external contributors) ---

# Dictionary to store basic details for each unique person
# Key: login (str), Value: {'name': str, 'access': str}
perso_base_info = {}

# First, add all collaborators with 'yes' access status
# This also populates their names
collaborator_logins = set()
for coll in collaborators:
    collaborator_logins.add(coll.login)
    perso_base_info[coll.login] = {
        'name': coll.name if coll.name else 'N/A',
        'access': 'yes'
    }

# Now, add all_contributors who are NOT collaborators with 'no' access status
for contr in all_contributors:
    if contr.login not in perso_base_info:
        # This person is a contributor but not a direct collaborator
        perso_base_info[contr.login] = {
            'name': contr.name if contr.name else 'N/A',
            'access': 'no'
        }
    else:
        # If a person is both a contributor and a collaborator, ensure their name is captured
        # in case the contributor object had a better name than the collaborator object
        if perso_base_info[contr.login]['name'] == 'N/A' and contr.name:
            perso_base_info[contr.login]['name'] = contr.name

# Convert this base info to a DataFrame
df_people = pd.DataFrame.from_dict(perso_base_info, orient='index')
df_people.index.name = 'github_login'
df_people = df_people.reset_index()

print("Unified DataFrame of Collaborators and External Contributors:")
display(df_people.head())

#print("\nDataFrame Info:")
#df_people.info()

Unified DataFrame of Collaborators and External Contributors:


,github_login,name,access
0,monacofj,Monaco F. J.,yes
1,LiviaLelis,Lívia Lelis,yes
2,trqt,Guilherme Torquato,yes
3,jotaefepinho,João Francisco Pinho,yes
4,chonkachu,N/A,yes


# FETCH EVENTS

In [26]:
## Confirm the calculated analysis timedelta.

from datetime import timedelta

# selected_timedelta is now calculated directly in the 'period' cell.
# This cell now just confirms the value.

print(f"Corresponding timedelta for analysis: {selected_timedelta}")

Corresponding timedelta for analysis: 28 days, 0:00:00


In [27]:
## Count core developers and external contributors.

num_core_devs = df_people[df_people['access'] == 'yes'].shape[0]
num_externals = df_people[df_people['access'] == 'no'].shape[0]

print(f"* Number of collaborators (core devs): {num_core_devs}")
print(f"* Number of contributors without repo access (externals): {num_externals}")

* Number of collaborators (core devs): 65
* Number of contributors without repo access (externals): 3


In [28]:
## Initialize dictionaries for daily event counts.

# Initialize dictionaries for daily event counts
daily_issues_raised_count = {}
daily_issue_comment_count = {}
daily_prs_opened_count = {}
daily_prs_merged_count = {}
daily_prs_closed_count = {}
daily_additions = {}
daily_deletions = {}

## Issues



In [29]:
## Fetch and count opened issues.

import os
from google.colab import userdata
from github import Github
from datetime import datetime, timedelta, timezone
import pandas as pd

# Calculate the start date based on the selected_timedelta
# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionary to store issues opened count per user
issues_opened_count = {}

# Fetch all issues since the start_date. This fetches issues CREATED or UPDATED since start_date.
issues = repo.get_issues(state='all', since=start_date)

for issue in issues:
    # Only count issues that were CREATED within the selected period
    if issue.user and issue.created_at >= start_date:
        login = issue.user.login
        issues_opened_count[login] = issues_opened_count.get(login, 0) + 1

        # NEW: Populate daily_issues_raised_count
        date_key = issue.created_at.date()
        daily_issues_raised_count[date_key] = daily_issues_raised_count.get(date_key, 0) + 1

# Convert the dictionary to a pandas Series for merging
issues_opened_series = pd.Series(issues_opened_count, name='issues_opened_count')

# --- Fix for KeyError: Drop existing 'issues_opened_count' related columns before merging ---
columns_to_drop = [col for col in df_people.columns if col.startswith('issues_opened_count')]
if columns_to_drop:
    df_people = df_people.drop(columns=columns_to_drop)

df_people = df_people.set_index('github_login')
df_people = df_people.merge(issues_opened_series, left_index=True, right_index=True, how='left')
df_people = df_people.reset_index()

# Fill NaN values with 0 for users who didn't open any issues in the period
df_people['issues_opened_count'] = df_people['issues_opened_count'].fillna(0).astype(int)

print("df_people DataFrame with 'issues_opened_count' column:")
display(df_people.head())

print("\nTop 4 people with most issues opened:")
top_issues_openers = df_people.sort_values(by='issues_opened_count', ascending=False).head(4)
for index, row in top_issues_openers.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['issues_opened_count']} issues opened")

df_people DataFrame with 'issues_opened_count' column:


,github_login,name,access,issues_opened_count
0,monacofj,Monaco F. J.,yes,1
1,LiviaLelis,Lívia Lelis,yes,2
2,trqt,Guilherme Torquato,yes,10
3,jotaefepinho,João Francisco Pinho,yes,1
4,chonkachu,N/A,yes,0



Top 4 people with most issues opened:
- trqt (Guilherme Torquato): 10 issues opened
- LeticiaBN (Leticia Barbosa Neves): 8 issues opened
- Nikk0o (Niko): 7 issues opened
- bdogabriel (Gabriel Barbosa): 6 issues opened


In [30]:
## Fetch and count reopened issues.

from datetime import datetime, timezone
import pandas as pd

# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionary to store issues reopened count per user
issues_reopened_count = {}

# Fetch all issues that were created or updated since the start_date
# This gives us a pool of issues that might have been reopened in the period
issues_in_period = repo.get_issues(state='all', since=start_date)

for issue in issues_in_period:
    try:
        # Get events for this specific issue (timeline events)
        issue_timeline_events = issue.get_events()
        for event in issue_timeline_events:
            # Check if the event is a 'reopened' event, happened in the period, and has an actor
            if event.event == 'reopened' and event.created_at >= start_date and event.actor:
                login = event.actor.login
                issues_reopened_count[login] = issues_reopened_count.get(login, 0) + 1

                # NEW: Populate daily_issues_raised_count
                date_key = event.created_at.date()
                daily_issues_raised_count[date_key] = daily_issues_raised_count.get(date_key, 0) + 1
    except Exception as e:
        # Handle potential errors if issue events cannot be fetched (e.g., rate limits, deleted events)
        print(f"Warning: Could not fetch events for issue #{issue.number}: {e}")

# Convert the dictionary to a pandas Series for merging
issues_reopened_series = pd.Series(issues_reopened_count, name='issues_reopened_count')

# Merge the counts into df_people
df_people = df_people.set_index('github_login')
df_people = df_people.merge(issues_reopened_series, left_index=True, right_index=True, how='left')
df_people = df_people.reset_index()

# Fill NaN values with 0 for users who didn't reopen any issues in the period
df_people['issues_reopened_count'] = df_people['issues_reopened_count'].fillna(0).astype(int)

print("df_people DataFrame with 'issues_reopened_count' column:")
display(df_people.head())

print("\nTop 4 people with most issues reopened:")
top_reopeners = df_people.sort_values(by='issues_reopened_count', ascending=False).head(4)
for index, row in top_reopeners.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['issues_reopened_count']} issues reopened")

df_people DataFrame with 'issues_reopened_count' column:


,github_login,name,access,issues_opened_count,issues_reopened_count
0,monacofj,Monaco F. J.,yes,1,0
1,LiviaLelis,Lívia Lelis,yes,2,0
2,trqt,Guilherme Torquato,yes,10,7
3,jotaefepinho,João Francisco Pinho,yes,1,0
4,chonkachu,N/A,yes,0,0



Top 4 people with most issues reopened:
- trqt (Guilherme Torquato): 7 issues reopened
- LeticiaBN (Leticia Barbosa Neves): 2 issues reopened
- otaviobia (Otávio Biagioni): 1 issues reopened
- goll72 (goll): 1 issues reopened


## Comments

PR are treated as issues with code.

In [31]:
## Fetch and count issue and PR comments.

from datetime import datetime, timezone
import pandas as pd

# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionary to store IssueCommentEvent counts per user
issue_comment_event_counts = {}

# --- Process Issue Comments ---
# Fetch all issues since the start_date (this method supports 'since')
issues_in_period = repo.get_issues(state='all', since=start_date)

for issue in issues_in_period:
    # Fetch comments for each issue
    comments = issue.get_comments()
    for comment in comments:
        # Ensure the comment itself was created within the period and has a user
        if comment.user and comment.created_at >= start_date:
            login = comment.user.login
            issue_comment_event_counts[login] = issue_comment_event_counts.get(login, 0) + 1

            # NEW: Populate daily_issue_comment_count
            date_key = comment.created_at.date()
            daily_issue_comment_count[date_key] = daily_issue_comment_count.get(date_key, 0) + 1

# --- Process Pull Request Comments (which are also IssueCommentEvent conceptually) ---
# Fetch all pull requests (this method does NOT support 'since')
pulls_in_period = repo.get_pulls(state='all')

for pull in pulls_in_period:
    # Fetch comments for each pull request
    comments = pull.get_comments()
    for comment in comments:
        # Ensure the comment itself was created within the period and has a user
        if comment.user and comment.created_at >= start_date:
            login = comment.user.login
            issue_comment_event_counts[login] = issue_comment_event_counts.get(login, 0) + 1

            # NEW: Populate daily_issue_comment_count
            date_key = comment.created_at.date()
            daily_issue_comment_count[date_key] = daily_issue_comment_count.get(date_key, 0) + 1

# Convert the dictionary to a pandas Series for merging
issue_comment_event_series = pd.Series(issue_comment_event_counts, name='issue_comment_event_count')

# Merge the counts into df_people
df_people = df_people.set_index('github_login')
df_people = df_people.merge(issue_comment_event_series, left_index=True, right_index=True, how='left')
df_people = df_people.reset_index()

# Fill NaN values with 0 for users who didn't perform any IssueCommentEvent in the period
df_people['issue_comment_event_count'] = df_people['issue_comment_event_count'].fillna(0).astype(int)

print("df_people DataFrame with 'issue_comment_event_count' column:")
display(df_people.head())

print("\nTop 4 people with most IssueCommentEvents:")
top_commenters = df_people.sort_values(by='issue_comment_event_count', ascending=False).head(4)
for index, row in top_commenters.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['issue_comment_event_count']} IssueCommentEvents")

df_people DataFrame with 'issue_comment_event_count' column:


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count
0,monacofj,Monaco F. J.,yes,1,0,0
1,LiviaLelis,Lívia Lelis,yes,2,0,8
2,trqt,Guilherme Torquato,yes,10,7,18
3,jotaefepinho,João Francisco Pinho,yes,1,0,2
4,chonkachu,N/A,yes,0,0,0



Top 4 people with most IssueCommentEvents:
- davigdomingues (Davi Gabriel Domingues): 29 IssueCommentEvents
- clr-cera (clr): 27 IssueCommentEvents
- trqt (Guilherme Torquato): 18 IssueCommentEvents
- DeguShi (Felipe Azambuja): 14 IssueCommentEvents


## Pull requests

In [32]:
## Fetch and count PRs opened, merged, and closed.

from datetime import datetime, timezone
import pandas as pd

# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionary to store pull requests opened count per user
pull_requests_opened_count = {}
# Dictionary to store pull requests merged count per user
pull_requests_merged_count = {}
# Dictionary to store pull requests closed (including merged) count per user
pull_requests_closed_count = {}

# Fetch all pull requests (this method does NOT support 'since' for 'all' state directly, but can filter later)
# It's generally better to iterate through all and filter by date.
all_pulls = repo.get_pulls(state='all')

for pull in all_pulls:
    # Only count pull requests that were CREATED within the selected period
    if pull.user and pull.created_at >= start_date:
        login = pull.user.login
        pull_requests_opened_count[login] = pull_requests_opened_count.get(login, 0) + 1

        # NEW: Populate daily_prs_opened_count
        date_key = pull.created_at.date()
        daily_prs_opened_count[date_key] = daily_prs_opened_count.get(date_key, 0) + 1

    # Count pull requests that were MERGED within the selected period
    if pull.merged and pull.merged_at and pull.merged_at >= start_date:
        if pull.merged_by:
            login = pull.merged_by.login
            pull_requests_merged_count[login] = pull_requests_merged_count.get(login, 0) + 1

            # NEW: Populate daily_prs_merged_count
            date_key = pull.merged_at.date()
            daily_prs_merged_count[date_key] = daily_prs_merged_count.get(date_key, 0) + 1

    # Count pull requests that were CLOSED (either merged or closed without merging) within the selected period
    # Note: 'closed_at' is typically populated for both merged and explicitly closed PRs.
    # We attribute the closure to the user who opened the PR for PI calculation.
    if pull.user and pull.state == 'closed' and pull.closed_at and pull.closed_at >= start_date:
        login = pull.user.login # User who created the PR
        pull_requests_closed_count[login] = pull_requests_closed_count.get(login, 0) + 1

        # NEW: Populate daily_prs_closed_count
        date_key = pull.closed_at.date()
        daily_prs_closed_count[date_key] = daily_prs_closed_count.get(date_key, 0) + 1

# Convert the dictionaries to pandas Series for merging
pull_requests_opened_series = pd.Series(pull_requests_opened_count, name='pull_requests_opened_count')
pull_requests_merged_series = pd.Series(pull_requests_merged_count, name='pull_requests_merged_count')
pull_requests_closed_series = pd.Series(pull_requests_closed_count, name='pull_requests_closed_count')


# --- Drop existing 'pull_requests_opened_count', 'pull_requests_merged_count', 'pull_requests_closed_count' related columns before merging ---
columns_to_drop_opened = [col for col in df_people.columns if col.startswith('pull_requests_opened_count')]
if columns_to_drop_opened:
    df_people = df_people.drop(columns=columns_to_drop_opened)

columns_to_drop_merged = [col for col in df_people.columns if col.startswith('pull_requests_merged_count')]
if columns_to_drop_merged:
    df_people = df_people.drop(columns=columns_to_drop_merged)

columns_to_drop_closed = [col for col in df_people.columns if col.startswith('pull_requests_closed_count')]
if columns_to_drop_closed:
    df_people = df_people.drop(columns=columns_to_drop_closed)


df_people = df_people.set_index('github_login')

# Merge counts
df_people = df_people.merge(pull_requests_opened_series, left_index=True, right_index=True, how='left')
df_people = df_people.merge(pull_requests_merged_series, left_index=True, right_index=True, how='left')
df_people = df_people.merge(pull_requests_closed_series, left_index=True, right_index=True, how='left')

df_people = df_people.reset_index()

# Fill NaN values with 0
df_people['pull_requests_opened_count'] = df_people['pull_requests_opened_count'].fillna(0).astype(int)
df_people['pull_requests_merged_count'] = df_people['pull_requests_merged_count'].fillna(0).astype(int)
df_people['pull_requests_closed_count'] = df_people['pull_requests_closed_count'].fillna(0).astype(int)

print("df_people DataFrame with PR counts:")
display(df_people.head())

print("\nTop 4 people with most Pull Requests opened:")
top_pr_openers = df_people.sort_values(by='pull_requests_opened_count', ascending=False).head(4)
for index, row in top_pr_openers.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['pull_requests_opened_count']} PRs opened")

print("\nTop 4 people with most Pull Requests merged:")
top_pr_mergers = df_people.sort_values(by='pull_requests_merged_count', ascending=False).head(4)
for index, row in top_pr_mergers.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['pull_requests_merged_count']} PRs merged")

print("\nTop 4 people with most Pull Requests closed:")
top_pr_closed = df_people.sort_values(by='pull_requests_closed_count', ascending=False).head(4)
for index, row in top_pr_closed.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['pull_requests_closed_count']} PRs closed")

df_people DataFrame with PR counts:


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count,pull_requests_opened_count,pull_requests_merged_count,pull_requests_closed_count
0,monacofj,Monaco F. J.,yes,1,0,0,0,0,0
1,LiviaLelis,Lívia Lelis,yes,2,0,8,1,1,1
2,trqt,Guilherme Torquato,yes,10,7,18,4,4,6
3,jotaefepinho,João Francisco Pinho,yes,1,0,2,0,0,2
4,chonkachu,N/A,yes,0,0,0,0,0,0



Top 4 people with most Pull Requests opened:
- LeticiaBN (Leticia Barbosa Neves): 5 PRs opened
- trqt (Guilherme Torquato): 4 PRs opened
- bdogabriel (Gabriel Barbosa): 4 PRs opened
- DeguShi (Felipe Azambuja): 3 PRs opened

Top 4 people with most Pull Requests merged:
- LeticiaBN (Leticia Barbosa Neves): 7 PRs merged
- clr-cera (clr): 6 PRs merged
- trqt (Guilherme Torquato): 4 PRs merged
- DeguShi (Felipe Azambuja): 4 PRs merged

Top 4 people with most Pull Requests closed:
- LeticiaBN (Leticia Barbosa Neves): 7 PRs closed
- trqt (Guilherme Torquato): 6 PRs closed
- davigdomingues (Davi Gabriel Domingues): 5 PRs closed
- bdogabriel (Gabriel Barbosa): 4 PRs closed


## Code changes

In [33]:
## Fetch and count code additions and deletions by user.

from datetime import datetime, timezone
import pandas as pd

# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionaries to store additions and deletions per user
user_additions = {}
user_deletions = {}

print(f"Fetching code changes from merged and currently open Pull Requests since {start_date}...")

# Fetch all pull requests (open, closed, and all states to check merged status)
all_pulls = repo.get_pulls(state='all')

for pull in all_pulls:
    process_pull = False
    # Condition 1: PR was merged within the period
    if pull.merged and pull.merged_at and pull.merged_at >= start_date:
        process_pull = True
    # Condition 2: PR is currently open
    elif pull.state == 'open':
        process_pull = True

    if process_pull:
        # Iterate through the commits of the relevant PR
        try:
            pr_commits = pull.get_commits()
            for commit in pr_commits:
                # Check if the commit's author exists
                if commit.author and commit.author.login:
                    login = commit.author.login
                    try:
                        full_commit = repo.get_commit(commit.sha)
                        if full_commit.stats:
                            # NEW: Check if commit date is within the period
                            commit_datetime = full_commit.commit.author.date # This is already a timezone-aware datetime
                            # Ensure comparison is done with timezone-aware objects
                            if commit_datetime.astimezone(timezone.utc) >= start_date:
                                commit_date_key = commit_datetime.date() # Get only the date part

                                user_additions[login] = user_additions.get(login, 0) + full_commit.stats.additions
                                user_deletions[login] = user_deletions.get(login, 0) + full_commit.stats.deletions

                                # NEW: Populate daily_additions and daily_deletions
                                daily_additions[commit_date_key] = daily_additions.get(commit_date_key, 0) + full_commit.stats.additions
                                daily_deletions[commit_date_key] = daily_deletions.get(commit_date_key, 0) + full_commit.stats.deletions
                    except Exception as e:
                        print(f"Warning: Could not fetch stats for commit {commit.sha} in PR #{pull.number} by {login}: {e}")
        except Exception as e:
            print(f"Warning: Could not fetch commits for PR #{pull.number}: {e}")

# Convert dictionaries to pandas Series
additions_series = pd.Series(user_additions, name='total_additions')
deletions_series = pd.Series(user_deletions, name='total_deletions')

# --- Drop existing columns before merging ---
# This prevents duplicate columns if the cell is run multiple times
columns_to_drop_add = [col for col in df_people.columns if col.startswith('total_additions')]
if columns_to_drop_add:
    df_people = df_people.drop(columns=columns_to_drop_add)

columns_to_drop_del = [col for col in df_people.columns if col.startswith('total_deletions')]
if columns_to_drop_del:
    df_people = df_people.drop(columns=columns_to_drop_del)

df_people = df_people.set_index('github_login')

# Merge additions and deletions into df_people
df_people = df_people.merge(additions_series, left_index=True, right_index=True, how='left')
df_people = df_people.merge(deletions_series, left_index=True, right_index=True, how='left')

df_people = df_people.reset_index()

# Fill NaN values with 0 for users who had no additions or deletions in the period
df_people['total_additions'] = df_people['total_additions'].fillna(0).astype(int)
df_people['total_deletions'] = df_people['total_deletions'].fillna(0).astype(int)

print("\ndf_people DataFrame with 'total_additions' and 'total_deletions' columns:")
display(df_people.head())

print("\nTop 4 people with most additions:")
top_adders = df_people.sort_values(by='total_additions', ascending=False).head(4)
for index, row in top_adders.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['total_additions']} additions")

print("\nTop 4 people with most deletions:")
top_deleters = df_people.sort_values(by='total_deletions', ascending=False).head(4)
for index, row in top_deleters.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['total_deletions']} deletions")

Fetching code changes from merged and currently open Pull Requests since 2025-11-02 14:15:54.555897+00:00...

df_people DataFrame with 'total_additions' and 'total_deletions' columns:


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count,pull_requests_opened_count,pull_requests_merged_count,pull_requests_closed_count,total_additions,total_deletions
0,monacofj,Monaco F. J.,yes,1,0,0,0,0,0,0,0
1,LiviaLelis,Lívia Lelis,yes,2,0,8,1,1,1,1832,250
2,trqt,Guilherme Torquato,yes,10,7,18,4,4,6,486,29
3,jotaefepinho,João Francisco Pinho,yes,1,0,2,0,0,2,0,0
4,chonkachu,N/A,yes,0,0,0,0,0,0,0,0



Top 4 people with most additions:
- DeguShi (Felipe Azambuja): 7471 additions
- MatheusPaivaa (Matheus Paiva): 3223 additions
- davigdomingues (Davi Gabriel Domingues): 2084 additions
- LiviaLelis (Lívia Lelis): 1832 additions

Top 4 people with most deletions:
- clr-cera (clr): 3261 deletions
- DeguShi (Felipe Azambuja): 1881 deletions
- LeticiaBN (Leticia Barbosa Neves): 488 deletions
- MatheusPaivaa (Matheus Paiva): 395 deletions


In [34]:
# cellname : fix-count-scales

import pandas as pd
import numpy as np

# 2. Define a list named other_event_cols
other_event_cols = [
    'issues_opened_count',
    'issues_reopened_count',
    'issue_comment_event_count',
    'pull_requests_opened_count',
    'pull_requests_merged_count'
]

# Ensure all columns exist, filter out non-existent ones if any
existing_other_event_cols = [col for col in other_event_cols if col in df_people.columns]

# 3. Calculate max_other_event_count
if existing_other_event_cols:
    max_other_event_count = df_people[existing_other_event_cols].max().max()
else:
    max_other_event_count = 0

# Avoid division by zero if no other events
if max_other_event_count == 0:
    max_other_event_count = 1

# 4. Calculate max_total_additions and max_total_deletions
max_total_additions = df_people['total_additions'].max()
max_total_deletions = df_people['total_deletions'].max()

# Determine max_code_change as the maximum value between max_total_additions and max_total_deletions
max_code_change = max(max_total_additions, max_total_deletions)

# Calculate the scaling_ratio by dividing max_code_change by max_other_event_count
# Handle the case where max_other_event_count is zero to avoid division errors.
scaling_ratio = max_code_change / max_other_event_count if max_other_event_count > 0 else 0

# Based on scaling_ratio, determine the chosen_factor:
chosen_factor = 1 # Default value
if max_code_change == 0: # If there are no code changes, factor is 1
    chosen_factor = 1
elif scaling_ratio <= 1:
    chosen_factor = 1
elif 1 < scaling_ratio <= 10:
    chosen_factor = 10
elif scaling_ratio > 10 and scaling_ratio <= 100:
    chosen_factor = int(np.ceil(scaling_ratio / 10)) * 10
elif scaling_ratio > 100:
    chosen_factor = int(np.ceil(scaling_ratio / 100)) * 100

# Create a scaled_label_suffix string
if chosen_factor == 1:
    scaled_label_suffix = ''
else:
    scaled_label_suffix = f' (x{chosen_factor})'

# 5. Initialize new columns (or overwrite if they exist)
df_people['scaled_additions'] = 0.0
df_people['scaled_deletions'] = 0.0

# Update the scaled_additions column in df_people
df_people['scaled_additions'] = df_people['total_additions'] / chosen_factor

# Update the scaled_deletions column in df_people
df_people['scaled_deletions'] = df_people['total_deletions'] / chosen_factor

# 8. Print the calculated values
print(f"Maximum count of other event types (for scaling): {max_other_event_count}")
print(f"Maximum total additions: {max_total_additions}")
print(f"Maximum total deletions: {max_total_deletions}")
print(f"Maximum code change (additions or deletions): {max_code_change}")
if max_code_change > 0 and max_other_event_count > 0:
    print(f"Scaling ratio (max_code_change / max_other_event_count): {scaling_ratio:.4f}")
print(f"Chosen factor for scaling code changes: {chosen_factor}")
print(f"Scaled label suffix: '{scaled_label_suffix}'")

# 9. Display the head of the df_people DataFrame
print("\ndf_people DataFrame with scaled additions and deletions:")
display(df_people[['github_login', 'name', 'total_additions', 'scaled_additions', 'total_deletions', 'scaled_deletions']].head())

# 10. Print top 4 contributors by scaled_additions
print("\nTop 4 people by scaled additions:")
top_scaled_adders = df_people.sort_values(by='scaled_additions', ascending=False).head(4)
for index, row in top_scaled_adders.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['scaled_additions']:.2f} scaled additions")

# 11. Print top 4 contributors by scaled_deletions
print("\nTop 4 people by scaled deletions:")
top_scaled_deleters = df_people.sort_values(by='scaled_deletions', ascending=False).head(4)
for index, row in top_scaled_deleters.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['scaled_deletions']:.2f} scaled deletions")

Maximum count of other event types (for scaling): 29
Maximum total additions: 7471
Maximum total deletions: 3261
Maximum code change (additions or deletions): 7471
Scaling ratio (max_code_change / max_other_event_count): 257.6207
Chosen factor for scaling code changes: 300
Scaled label suffix: ' (x300)'

df_people DataFrame with scaled additions and deletions:


,github_login,name,total_additions,scaled_additions,total_deletions,scaled_deletions
0,monacofj,Monaco F. J.,0,0.000000,0,0.000000
1,LiviaLelis,Lívia Lelis,1832,6.106667,250,0.833333
2,trqt,Guilherme Torquato,486,1.620000,29,0.096667
3,jotaefepinho,João Francisco Pinho,0,0.000000,0,0.000000
4,chonkachu,N/A,0,0.000000,0,0.000000



Top 4 people by scaled additions:
- DeguShi (Felipe Azambuja): 24.90 scaled additions
- MatheusPaivaa (Matheus Paiva): 10.74 scaled additions
- davigdomingues (Davi Gabriel Domingues): 6.95 scaled additions
- LiviaLelis (Lívia Lelis): 6.11 scaled additions

Top 4 people by scaled deletions:
- clr-cera (clr): 10.87 scaled deletions
- DeguShi (Felipe Azambuja): 6.27 scaled deletions
- LeticiaBN (Leticia Barbosa Neves): 1.63 scaled deletions
- MatheusPaivaa (Matheus Paiva): 1.32 scaled deletions


# PROJECT OVERVIEW

## Events (totals)

* Show all events in the period (ommit devs that haven't generated events)

In [35]:
# cellname : show-all-events

import plotly.express as px
import pandas as pd

# Diagnostic: Print all columns in df_people before processing for the plot
print("Current columns in df_people before plotting:")
print(df_people.columns)

# Define the columns that represent events, NOW USING SCALED ADDITIONS AND DELETIONS
event_columns = [
    'issues_opened_count',
    'issues_reopened_count',
    'issue_comment_event_count',
    'pull_requests_opened_count',
    'pull_requests_merged_count',
    'scaled_additions',  # Use scaled additions
    'scaled_deletions'   # Use scaled deletions
]

# Ensure all event columns exist in df_people
existing_event_columns = [col for col in event_columns if col in df_people.columns]

# Filter out developers with no activity in any of the tracked events for cleaner visualization
df_plot = df_people[df_people[existing_event_columns].sum(axis=1) > 0].copy()

# Rename columns for better plot labels (optional, but good practice)
df_plot = df_plot.rename(columns={
    'issues_opened_count': 'Issues Opened',
    'issues_reopened_count': 'Issues Reopened',
    'issue_comment_event_count': 'Comments (Issues/PRs)',
    'pull_requests_opened_count': 'PRs Opened',
    'pull_requests_merged_count': 'PRs Merged',
    'scaled_additions': f'Lines Added{scaled_label_suffix}', # Updated label with suffix
    'scaled_deletions': f'Lines Removed{scaled_label_suffix}' # Updated label with suffix
})

# Melt the DataFrame to a long format suitable for stacked bar charts
# We use 'github_login' as the id_vars because we want one bar per developer
df_melted = df_plot.melt(id_vars=['github_login', 'name'], value_vars=[col for col in df_plot.columns if col in [f'Lines Added{scaled_label_suffix}', f'Lines Removed{scaled_label_suffix}', 'Issues Opened', 'Issues Reopened', 'Comments (Issues/PRs)', 'PRs Opened', 'PRs Merged']],
                         var_name='Event Type', value_name='Count')

# Create the stacked bar chart using Plotly Express
fig = px.bar(df_melted,
             x='github_login',
             y='Count',
             color='Event Type',
             title=f'Events per Contributor', # Updated title
             labels={'github_login': 'Contributor', 'Count': 'Event Count'},
             hover_name='name',
             hover_data={'Event Type': True, 'Count': ':.2f', 'github_login': False},
             text_auto='.2f') # Display text values automatically, formatted to 2 decimal places

fig.update_layout(xaxis_title='Contributor',
                  yaxis_title='Event Count',
                  barmode='stack',
                  legend_title='Event Type')

# Set text color to white for all traces and position text inside
fig.update_traces(textfont_color='white', textposition='inside')

# Optional: Adjust x-axis to show only relevant developer names if too many
# fig.update_xaxes(tickangle=45, tickfont=dict(size=10))

fig.show() # Always show the figure

# Save the figure to Google Drive if report generation is enabled
save_fig(fig, 'all_primitive_events.html')


Current columns in df_people before plotting:
Index(['github_login', 'name', 'access', 'issues_opened_count',
       'issues_reopened_count', 'issue_comment_event_count',
       'pull_requests_opened_count', 'pull_requests_merged_count',
       'pull_requests_closed_count', 'total_additions', 'total_deletions',
       'scaled_additions', 'scaled_deletions'],
      dtype='object')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/all_primitive_events.html


## Events categorized x time
* All events, but aggregating Issues, Commands,PRs and total changes

In [36]:
# cellname : time-plot
import pandas as pd
import plotly.express as px
from datetime import date, timedelta, datetime, timezone

# Ensure start_date and selected_timedelta are available (from previous cells)
# Recalculate full date range for the period
# It's important to use the same start_date as the event fetching to match the period.
# The start_date for event fetching is a datetime with timezone.
# We need to get the date part for comparison with dictionary keys.

# The start_date for fetching events is already timezone-aware
# start_date = datetime.now(timezone.utc) - selected_timedelta
# Let's ensure 'start_date' is available, it comes from bd46d806
if 'start_date' not in globals():
    start_date = datetime.now(timezone.utc) - selected_timedelta # Fallback

end_date_for_range = datetime.now(timezone.utc).date() # Current date
start_date_for_range = start_date.date() # Date part of the fetching start_date

all_dates = [start_date_for_range + timedelta(days=x) for x in range((end_date_for_range - start_date_for_range).days + 1)]

# Create a DataFrame to store daily events
df_daily_events = pd.DataFrame(all_dates, columns=['Date'])
df_daily_events['Date'] = pd.to_datetime(df_daily_events['Date']) # Convert to datetime objects

# Merge daily event counts
# Fillna(0) for dates where no events occurred
df_daily_events['Issues Raised'] = df_daily_events['Date'].apply(lambda d: daily_issues_raised_count.get(d.date(), 0))
df_daily_events['Comments'] = df_daily_events['Date'].apply(lambda d: daily_issue_comment_count.get(d.date(), 0))
df_daily_events['PRs Opened'] = df_daily_events['Date'].apply(lambda d: daily_prs_opened_count.get(d.date(), 0))
df_daily_events['PRs Merged'] = df_daily_events['Date'].apply(lambda d: daily_prs_merged_count.get(d.date(), 0))
df_daily_events['PRs Closed'] = df_daily_events['Date'].apply(lambda d: daily_prs_closed_count.get(d.date(), 0))

# Apply scaling to additions and deletions for consistency with other plots
# chosen_factor and scaled_label_suffix should be available from fix-count-scales (j-NJ5SHRtkyN)
if 'chosen_factor' not in globals():
    chosen_factor = 1 # Fallback if not defined (should be defined)
if 'scaled_label_suffix' not in globals():
    scaled_label_suffix = '' # Fallback

# Calculate scaled additions and deletions
df_daily_events[f'Lines Added{scaled_label_suffix}'] = df_daily_events['Date'].apply(lambda d: daily_additions.get(d.date(), 0) / chosen_factor)
df_daily_events[f'Lines Removed{scaled_label_suffix}'] = df_daily_events['Date'].apply(lambda d: daily_deletions.get(d.date(), 0) / chosen_factor)

# Combine 'Lines Added' and 'Lines Removed' into 'Lines Changed'
df_daily_events[f'Lines Changed{scaled_label_suffix}'] = df_daily_events[f'Lines Added{scaled_label_suffix}'] + df_daily_events[f'Lines Removed{scaled_label_suffix}']

# Melt the DataFrame for line chart visualization
df_daily_melted = df_daily_events.melt(
    id_vars=['Date'],
    value_vars=[
        'Issues Raised',
        'Comments',
        'PRs Opened',
        'PRs Merged',
        f'Lines Changed{scaled_label_suffix}' # Use combined category
    ],
    var_name='Event Type',
    value_name='Count'
)

# Create the line chart using Plotly Express with spline for smoothing
fig = px.line(
    df_daily_melted,
    x='Date',
    y='Count',
    color='Event Type',
    title='Categorized events over time', # Changed title here
    labels={'Count': 'Event Count', 'Date': 'Date'},
    hover_data={'Count': ':.2f'},
    line_shape='spline'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Count',
    legend_title='Event Type',
    hovermode='x unified' # Show all hover info for a given date
)

fig.update_xaxes(dtick="D1", tickformat="%b %d") # Daily ticks, format as "Month Day"

fig.show()

save_fig(fig, 'daily_events_time_series.html')

print("df_daily_events DataFrame (first 5 rows):")
display(df_daily_events.head())

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/daily_events_time_series.html
df_daily_events DataFrame (first 5 rows):


,Date,Issues Raised,Comments,PRs Opened,PRs Merged,PRs Closed,Lines Added (x300),Lines Removed (x300),Lines Changed (x300)
0,2025-11-02,0,0,0,0,0,0.000000,0.000000,0.000000
1,2025-11-03,0,0,0,0,0,0.000000,0.000000,0.000000
2,2025-11-04,11,13,6,1,1,2.060000,0.830000,2.890000
3,2025-11-05,2,0,1,0,0,0.333333,0.143333,0.476667
4,2025-11-06,0,0,0,0,0,0.000000,0.000000,0.000000


## Events categorized


In [37]:
# cellname : all-events-aggregate
import plotly.express as px
import pandas as pd

# Create consolidated event columns
df_consolidated = df_people.copy()
df_consolidated['Issues'] = df_consolidated['issues_opened_count'] + df_consolidated['issues_reopened_count']
df_consolidated['Comments'] = df_consolidated['issue_comment_event_count']
df_consolidated['PRs Opened'] = df_consolidated['pull_requests_opened_count']
df_consolidated['PRs Merged'] = df_consolidated['pull_requests_merged_count']

# Recalculate 'Code Changes' using scaled additions and deletions and include the suffix
df_consolidated[f'Code Changes{scaled_label_suffix}'] = df_consolidated['scaled_additions'] + df_consolidated['scaled_deletions']

# Define the new consolidated event columns for plotting, now including the scaled suffix
consolidated_event_columns = ['Issues', 'Comments', 'PRs Opened', 'PRs Merged', f'Code Changes{scaled_label_suffix}']

# Filter out developers with no activity in any of the consolidated events
df_plot_consolidated = df_consolidated[df_consolidated[consolidated_event_columns].sum(axis=1) > 0].copy()

# Melt the DataFrame to a long format suitable for stacked bar charts
df_melted_consolidated = df_plot_consolidated.melt(
    id_vars=['github_login', 'name'],
    value_vars=consolidated_event_columns,
    var_name='Event Type',
    value_name='Count'
)

# Rename 'Issues' to 'Issues Raised' for display in the plot legend
df_melted_consolidated['Event Type'] = df_melted_consolidated['Event Type'].replace('Issues', 'Issues Raised')

# Add a new column for text display, conditional on Count > 0 and formatted
df_melted_consolidated['text_values'] = df_melted_consolidated.apply(lambda row: f"{row['Count']:.2f}" if row['Count'] > 0 and row['Event Type'] == f'Code Changes{scaled_label_suffix}' else (f"{row['Count']:.0f}" if row['Count'] > 0 else ''), axis=1)


# Create the stacked bar chart using Plotly Express
fig_consolidated = px.bar(
    df_melted_consolidated,
    x='github_login',
    y='Count',
    color='Event Type',
    title=f'Categorized event count', # Updated title
    labels={'github_login': 'Contributor', 'Count': 'Event Count'},
    hover_name='name',
    hover_data={'Event Type': True, 'Count': ':.2f', 'github_login': False}, # Format Count to 2 decimal places
    text='text_values' # Use the new conditional text column
)

fig_consolidated.update_layout(
    xaxis_title='Contributor',
    yaxis_title='Event Count',
    barmode='stack',
    legend_title='Event Type'
)

# Adjust x-axis to show labels vertically
fig_consolidated.update_xaxes(tickangle=90, tickfont=dict(size=10), showgrid=False)

# Revert y-axis grid to default Plotly style, with lines every 5 units
fig_consolidated.update_yaxes(showgrid=True, showticklabels=True, dtick=5, gridwidth=1, griddash=None)

# Set text color to white for all traces and position text inside
fig_consolidated.update_traces(textfont_color='white', textposition='inside')

fig_consolidated.show()

save_fig(fig_consolidated, 'all_aggregate_events.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/all_aggregate_events.html


## Workload balance (WB) index

### WB aggregate

In [38]:
# cellname : wbi
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Calculate aggregate PRs opened and closed for the entire project
total_prs_opened = df_people['pull_requests_opened_count'].sum()
total_prs_closed = df_people['pull_requests_closed_count'].sum()

print(f"Total PRs Opened in the period: {total_prs_opened}")
print(f"Total PRs Closed in the period: {total_prs_closed}")

# --- Calculate the NEW Aggregate Progress Index (PI) (formerly PR Balance Index) ---
# PI = (PRs Closed - PRs Opened) / (PRs Closed + PRs Opened)
# Handle case where total_pr_activity is 0 to avoid division by zero
total_pr_activity = total_prs_opened + total_prs_closed

if total_pr_activity > 0:
    # The new PI is the former PR Balance Index
    wb_index = (total_prs_closed - total_prs_opened) / total_pr_activity
    print(f"Work Balance (WB): {wb_index:.2f}\n")
else:
    wb_index = 0.0 # Define a default if no PRs were opened and closed
    print("No PR activity (opened or closed) in the period to calculate the Net Contribution Throughput (NCT).\n")

# --- Horizontal Bar Chart for Net Contribution Throughput (NCT) ---

# Determine marker color dynamically
#bar_color_pi = '#AEC6CF'
if wb_index < 0:
    bar_color_pi = "#dd5500"
elif wb_index > 0:
    bar_color_pi = "#00aa88"

fig_horizontal_bar = go.Figure()

# Add background shapes for the ranges (reverting y0 and y1 for taller colored segments)
fig_horizontal_bar.update_layout(
    shapes=[
        # More vivid Red for [-1, -0.5]
        dict(type='rect', xref='x', yref='y', x0=-1, y0=-0.5, x1=-0.5, y1=0.5, fillcolor='#FF6347', opacity=0.3, layer='below', line_width=0),
        # More vivid Salmon for [-0.5, 0]
        dict(type='rect', xref='x', yref='y', x0=-0.5, y0=-0.5, x1=0, y1=0.5, fillcolor='#FFA07A', opacity=0.3, layer='below', line_width=0),
        # More vivid Light Green for [0, 0.5]
        dict(type='rect', xref='x', yref='y', x0=0, y0=-0.5, x1=0.5, y1=0.5, fillcolor='#90EE90', opacity=0.3, layer='below', line_width=0),
        # More vivid Darker Green for [0.5, 1]
        dict(type='rect', xref='x', yref='y', x0=0.5, y0=-0.5, x1=1, y1=0.5, fillcolor='#3CB371', opacity=0.3, layer='below', line_width=0)
    ]
)

fig_horizontal_bar.add_trace(go.Bar(
    x=[wb_index],
    y=['NCT'],
    orientation='h',
    base=0,
    marker_color=bar_color_pi,
    text=f'{wb_index:.2f}',
    textposition='inside',
    textfont=dict(color='white'),
    width=0.4 # Explicitly setting the width (height for horizontal bar) of the blue indicator bar
))

fig_horizontal_bar.update_layout(
    title_text='Workload Balance (WB)', # Updated title
    xaxis_range=[-1, 1], # Set x-axis range strictly from -1 to 1
    xaxis_title='← Accumulating | Clearing →', # Updated x-axis title
    yaxis_visible=False, # Hide y-axis
    yaxis_showticklabels=False, # Hide y-axis tick labels
    height=150, # Set a reasonable height for the bar chart
    width=800, # Set a reasonable width
    margin=dict(l=20, r=20, t=50, b=20), # Adjust margins
    title_x=0.5 # Center the title
)

fig_horizontal_bar.show()

save_fig(fig_horizontal_bar, 'workload_balance.html')


Total PRs Opened in the period: 62
Total PRs Closed in the period: 91
Work Balance (WB): 0.19



Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/workload_balance.html


### WB evolution

In [39]:
## Calculate and plot daily Workload Balance (WB) evolution.

import pandas as pd
import plotly.express as px
from datetime import date, timedelta, datetime, timezone

# Ensure start_date and selected_timedelta are available
if 'start_date' not in globals():
    start_date = datetime.now(timezone.utc) - timedelta(weeks=1) # Fallback to 1 week if not defined

end_date_for_range = datetime.now(timezone.utc).date() # Current date
start_date_for_range = start_date.date() # Date part of the fetching start_date

all_dates = [start_date_for_range + timedelta(days=x) for x in range((end_date_for_range - start_date_for_range).days + 1)]

daily_wb_data = []

for current_date in all_dates:
    # Get daily PRs opened and closed from the dictionaries
    daily_opened = daily_prs_opened_count.get(current_date, 0)
    daily_closed = daily_prs_closed_count.get(current_date, 0)

    # Calculate daily WB
    total_daily_pr_activity = daily_opened + daily_closed
    if total_daily_pr_activity > 0:
        daily_wb = (daily_closed - daily_opened) / total_daily_pr_activity
    else:
        daily_wb = 0.0 # No PR activity for the day

    daily_wb_data.append({'Date': current_date, 'Daily WB': daily_wb})

df_daily_wb = pd.DataFrame(daily_wb_data)
df_daily_wb['Date'] = pd.to_datetime(df_daily_wb['Date'])

# Create the smoothed line chart for daily WB
fig = px.line(
    df_daily_wb,
    x='Date',
    y='Daily WB',
    title='Daily Workload Balance (WB) over Time',
    labels={'Daily WB': 'Workload Balance (WB)', 'Date': 'Date'},
    hover_data={'Daily WB': ':.2f'},
    line_shape='spline'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Workload Balance (WB)',
    hovermode='x unified',
    yaxis_range=[-1.1, 1.1]
)

fig.update_xaxes(dtick="D1", tickformat="%b %d") # Daily ticks, format as "Month Day"

# Get the last date for the arrow placement
last_date = df_daily_wb['Date'].iloc[-1]

# Add an annotation for the 'Clearing' arrow (pointing up)
fig.add_annotation(
    x=last_date,
    y=0.75,  # Arrowhead at y=0.75 (shortened length)
    ax=last_date,
    ay=0.05,    # Arrow tail slightly above y=0
    showarrow=True,
    arrowhead=3,
    arrowsize=1.5,
    arrowwidth=1,
    arrowcolor='green',
    text="", # No text for the arrow itself
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    xshift=10 # Shift arrow slightly to the right
)

# Add a separate annotation for the text "Clearing"
fig.add_annotation(
    x=last_date,
    y=0.95,  # Position the text higher, near y=1
    text='Clearing',
    showarrow=False, # No arrow for the text
    yanchor='bottom', # Anchor the bottom of the text to its y-position
    font=dict(color='green'), # Set text color to green
    xref='x',
    yref='y',
    xshift=10 # Shift text slightly to the right
)

# Add an annotation for the 'Accumulating' arrow (pointing down)
fig.add_annotation(
    x=last_date,
    y=-0.75, # Arrowhead at y=-0.75
    ax=last_date,
    ay=-0.05, # Arrow tail slightly below y=0
    showarrow=True,
    arrowhead=3,
    arrowsize=1.5,
    arrowwidth=1,
    arrowcolor='red',
    text="", # No text for the arrow itself
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    xshift=10 # Shift arrow slightly to the right
)

# Add a separate annotation for the text "Accumulating"
fig.add_annotation(
    x=last_date,
    y=-0.85, # Position the text below the arrow head (-0.75)
    text='Accumulating',
    showarrow=False, # No arrow for the text
    yanchor='top', # Anchor the top of the text to its y-position
    font=dict(color='red'), # Set text color to red
    xref='x',
    yref='y',
    xshift=10 # Shift text slightly to the right
)

fig.show()
save_fig(fig, 'daily_wb_line_chart.html')

print("Daily Workload Balance (WB) chart generated.")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/daily_wb_line_chart.html
Daily Workload Balance (WB) chart generated.


## Resolution balance (RB) index

### RB aggregate

In [40]:
# cellname : ebi

import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Calculate total issues raised (opened + reopened)
total_issues_raised = df_people['issues_opened_count'].sum() + df_people['issues_reopened_count'].sum()

# Get total PRs submitted (using 'pull_requests_opened_count' as proxy for submitted)
total_prs_submitted = df_people['pull_requests_opened_count'].sum()

print(f"Total Issues Raised in the period: {total_issues_raised}")
print(f"Total PRs Submitted in the period: {total_prs_submitted}")

# --- Calculate the Submitted Index (SI) ---
# SI = (PRs Submitted - Issues Raised) / (PRs Submitted + Issues Raised)
# Handle case where total_activity is 0 to avoid division by zero
total_activity_for_si = total_prs_submitted + total_issues_raised

if total_activity_for_si > 0:
    rb_index = (total_prs_submitted - total_issues_raised) / total_activity_for_si
    print(f"Resolution Balance (RB): {rb_index:.2f}\n")
else:
    rb_index = 0.0 # Define a default if no relevant activity
    print("No PRs submitted or issues raised in the period to calculate the Submitted Index (SI).\n")

# --- Horizontal Bar Chart for Submitted Index (SI) ---

# Determine indicator bar color based on index value, as originally designed
if rb_index < 0:
    bar_color_si = "#888800"
elif rb_index > 0:
    bar_color_si = "#0088aa"
else:
    bar_color_si = "#AEC6CF" # Default grayish-blue if index is 0

fig_submitted_bar = go.Figure()

# Add background shapes for the ranges (reusing vivid colors from previous notebook state)
fig_submitted_bar.update_layout(
    shapes=[
        # More vivid Red for [-1, -0.5] -> Changed to Yellow
        dict(type='rect', xref='x', yref='y', x0=-1, y0=-0.5, x1=-0.5, y1=0.5, fillcolor='#FFDD33', opacity=0.3, layer='below', line_width=0),
        # More vivid Salmon for [-0.5, 0] -> Changed to Lighter Yellow
        dict(type='rect', xref='x', yref='y', x0=-0.5, y0=-0.5, x1=0, y1=0.5, fillcolor='#FFEE99', opacity=0.3, layer='below', line_width=0),
        # More vivid Light Green for [0, 0.5] -> Changed to Light Blue
        dict(type='rect', xref='x', yref='y', x0=0, y0=-0.5, x1=0.5, y1=0.5, fillcolor='#ADD8E6', opacity=0.3, layer='below', line_width=0),
        # More vivid Darker Green for [0.5, 1] -> Changed to Darker Blue
        dict(type='rect', xref='x', yref='y', x0=0.5, y0=-0.5, x1=1, y1=0.5, fillcolor='#4682B4', opacity=0.3, layer='below', line_width=0)
    ]
)

fig_submitted_bar.add_trace(go.Bar(
    x=[rb_index],
    y=['SI'],
    orientation='h',
    base=0,
    marker_color=bar_color_si,
    text=f'{rb_index:.2f}',
    textposition='inside',
    textfont=dict(color='white'), # White text color
    width=0.4 # Height of the indicator bar
))

fig_submitted_bar.update_layout(
    title_text='Resolution Balance (RB)', # Updated title
    xaxis_range=[-1, 1], # Set x-axis range strictly from -1 to 1
    xaxis_title='← Planning | Implementing →', # Updated x-axis title
    yaxis_visible=False, # Hide y-axis
    yaxis_showticklabels=False, # Hide y-axis tick labels
    height=150, # Set a reasonable height for the bar chart
    width=800, # Set a reasonable width
    margin=dict(l=20, r=20, t=50, b=20), # Adjust margins
    title_x=0.5 # Center the title
)

fig_submitted_bar.show()

save_fig(fig_submitted_bar, 'resolution_balance.html')


Total Issues Raised in the period: 148
Total PRs Submitted in the period: 62
Resolution Balance (RB): -0.41



Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/resolution_balance.html


### RB evolution

In [41]:
## Calculate and plot daily Resolution Balance (RB) evolution.

import pandas as pd
import plotly.express as px
from datetime import date, timedelta, datetime, timezone

# Ensure start_date and selected_timedelta are available (from previous cells)
if 'start_date' not in globals():
    start_date = datetime.now(timezone.utc) - timedelta(weeks=1) # Fallback to 1 week if not defined

end_date_for_range = datetime.now(timezone.utc).date() # Current date
start_date_for_range = start_date.date() # Date part of the fetching start_date

all_dates = [start_date_for_range + timedelta(days=x) for x in range((end_date_for_range - start_date_for_range).days + 1)]

daily_rb_data = []

for current_date in all_dates:
    # Get daily issues raised and PRs opened from the dictionaries
    daily_issues_raised = daily_issues_raised_count.get(current_date, 0)
    daily_prs_opened = daily_prs_opened_count.get(current_date, 0)

    # Calculate daily RB
    total_daily_activity_for_rb = daily_issues_raised + daily_prs_opened
    if total_daily_activity_for_rb > 0:
        daily_rb = (daily_prs_opened - daily_issues_raised) / total_daily_activity_for_rb
    else:
        daily_rb = 0.0 # No relevant activity for the day

    daily_rb_data.append({'Date': current_date, 'Daily RB': daily_rb})

df_daily_rb = pd.DataFrame(daily_rb_data)
df_daily_rb['Date'] = pd.to_datetime(df_daily_rb['Date'])

# Create the smoothed line chart for daily RB
fig = px.line(
    df_daily_rb,
    x='Date',
    y='Daily RB',
    title='Daily Resolution Balance (RB) over Time',
    labels={'Daily RB': 'Resolution Balance (RB)', 'Date': 'Date'},
    hover_data={'Daily RB': ':.2f'},
    line_shape='spline'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Resolution Balance (RB)',
    hovermode='x unified',
    yaxis_range=[-1.1, 1.1]
)

fig.update_xaxes(dtick="D1", tickformat="%b %d") # Daily ticks, format as "Month Day"

# Get the last date for the arrow placement
last_date = df_daily_rb['Date'].iloc[-1]

# Add an annotation for the 'Implementing' arrow (pointing up)
fig.add_annotation(
    x=last_date,
    y=0.75,  # Arrowhead at y=0.75
    ax=last_date,
    ay=0.05,    # Arrow tail slightly above y=0
    showarrow=True,
    arrowhead=3,
    arrowsize=1.5,
    arrowwidth=1,
    arrowcolor='blue', # Blue for implementing
    text="", # No text for the arrow itself
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    xshift=10 # Shift arrow slightly to the right
)

# Add a separate annotation for the text "Implementing"
fig.add_annotation(
    x=last_date,
    y=0.95,  # Position the text higher, near y=1
    text='Implementing',
    showarrow=False, # No arrow for the text
    yanchor='bottom', # Anchor the bottom of the text to its y-position
    font=dict(color='blue'), # Set text color to blue
    xref='x',
    yref='y',
    xshift=10 # Shift text slightly to the right
)

# Add an annotation for the 'Planning' arrow (pointing down)
fig.add_annotation(
    x=last_date,
    y=-0.75, # Arrowhead at y=-0.75
    ax=last_date,
    ay=-0.05, # Arrow tail slightly below y=0
    showarrow=True,
    arrowhead=3,
    arrowsize=1.5,
    arrowwidth=1,
    arrowcolor='orange', # Orange for planning
    text="", # No text for the arrow itself
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    xshift=10 # Shift arrow slightly to the right
)

# Add a separate annotation for the text "Planning"
fig.add_annotation(
    x=last_date,
    y=-0.85, # Position the text below the arrow head (-0.75)
    text='Planning',
    showarrow=False, # No arrow for the text
    yanchor='top', # Anchor the top of the text to its y-position
    font=dict(color='orange'), # Set text color to orange
    xref='x',
    yref='y',
    xshift=10 # Shift text slightly to the right
)

fig.show()
save_fig(fig, 'daily_rb_line_chart.html')

print("Daily Resolution Balance (RB) chart generated.")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/daily_rb_line_chart.html
Daily Resolution Balance (RB) chart generated.


## Develoment tide chart : WB x RB

In [42]:
## Plot aggregate WB vs RB on a polar chart.

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# 2. Create a Pandas DataFrame named df_metrics with 'EB' (submitted_index) and 'WB' (aggregate_pi)
# Ensure rb_index and wb_index are available from previous cells
df_metrics = pd.DataFrame({
    'RB': [rb_index],
    'WB': [wb_index]
})

# 1. Calculate the polar coordinates r (radius) and theta_degrees (angle in degrees)
#    from the EB and WB values in df_metrics.
df_metrics['r'] = np.sqrt(df_metrics['RB']**2 + df_metrics['WB']**2)
df_metrics['theta_degrees'] = np.degrees(np.arctan2(df_metrics['RB'], df_metrics['WB']))

# 2. Initialize the figure
fig = go.Figure()

# Add a separate trace for the line from the origin to the point (added first to be in the background)
fig.add_trace(go.Scatterpolar(
    r=[0, df_metrics['r'].iloc[0]], # Draw line from origin (r=0) to the point's r
    theta=[df_metrics['theta_degrees'].iloc[0], df_metrics['theta_degrees'].iloc[0]], # Maintain same angle
    mode='lines',
    line=dict(width=1, color='black', dash='dot'), # Updated line style: thin, black, dotted
    name='Handle from Origin'
))

# Add the trace for the marker and its text (added second to be on top)
fig.add_trace(go.Scatterpolar(
    r=df_metrics['r'],
    theta=df_metrics['theta_degrees'],
    mode='markers+text', # Restored mode to markers+text
    marker=dict(symbol='circle-open-dot', size=10, color='black', line=dict(width=2, color='black')),
    text=[f'RB: {df_metrics['RB'].iloc[0]:.2f}<br>WB: {df_metrics['WB'].iloc[0]:.2f}'],
    textposition='top center',
    name='Current State'
))

# 3. Update the layout of the figure to configure the polar axes
fig.update_layout(
    title_text='Project tide chart (WB x RB)', # Updated title
    title_x=0.5,
    height=600,
    width=800,
    polar=dict(
        radialaxis=dict(range=[0, np.sqrt(2) * 1.05]), # Adjusted range to accommodate sqrt(2) and a bit more for padding
        angularaxis=dict(
            rotation=90, # To align 0 degrees with 'Up' (+WB)
            direction='clockwise', # For clockwise angle increase
            tickvals=[0, 90, 180, 270],
            ticktext=['Clearing (+WB)', 'Implementing (+RB)', 'Accumulating (-WB)', 'Planning (-RB)']
        )
    ),
    showlegend=False # Hide legend as it's a single point and handle
)

# 6. Display the plot
fig.show()

save_fig(fig, 'wb_rb_map.html')

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/wb_rb_map.html


In [43]:
# cellname : wb-rb-time
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import date, timedelta, datetime, timezone

# Ensure start_date and selected_timedelta are available (from previous cells)
if 'start_date' not in globals():
    start_date = datetime.now(timezone.utc) - timedelta(weeks=1) # Fallback to 1 week if not defined

end_date_for_range = datetime.now(timezone.utc).date() # Current date
start_date_for_range = start_date.date() # Date part of the fetching start_date

all_dates = [start_date_for_range + timedelta(days=x) for x in range((end_date_for_range - start_date_for_range).days + 1)]

daily_metrics_wb_rb = []

for current_date in all_dates:
    # Get daily WB and RB from the dataframes (assume df_daily_wb and df_daily_rb are available)
    # Ensure current_date is compared with the date part of the DataFrame's 'Date' column
    daily_wb_val = df_daily_wb.loc[df_daily_wb['Date'].dt.date == current_date, 'Daily WB'].iloc[0] if current_date in df_daily_wb['Date'].dt.date.values else 0.0
    daily_rb_val = df_daily_rb.loc[df_daily_rb['Date'].dt.date == current_date, 'Daily RB'].iloc[0] if current_date in df_daily_rb['Date'].dt.date.values else 0.0

    # Calculate norm (r) and normalize it by sqrt(2) to be in [0, 1] range
    r_raw = np.sqrt(daily_wb_val**2 + daily_rb_val**2)
    norm_normalized = r_raw / np.sqrt(2) if np.sqrt(2) > 0 else 0.0

    # Calculate standard angle (theta) in degrees using np.arctan2(y, x)
    standard_angle_degrees = np.degrees(np.arctan2(daily_rb_val, daily_wb_val))

    # --- Apply Plotly's angular axis transformation to match polar diagram's display ---
    # This transformation converts the standard [-180, 180] angle to Plotly's [0, 360] scale
    # with rotation=90 and direction='clockwise'.
    # The formula is (standard_angle_0_360 - 90 + 360) % 360
    # First, convert standard_angle_degrees to [0, 360) range
    standard_angle_0_360 = (standard_angle_degrees + 360) % 360

    # Then apply the transformation for Plotly's axis (rotation=90, clockwise direction)
    # Plotly's 0 is standard 90
    # Plotly's 90 is standard 180
    # Plotly's 180 is standard 270 (or -90)
    # Plotly's 270 is standard 0
    # The transformation is (standard_angle_0_360 - 90 + 360) % 360 to map to the tickvals.
    # Let's verify: std 0 -> (0-90+360)%360 = 270 (Clearing is at 270). OK
    #              std 90 -> (90-90+360)%360 = 0 (Implementing is at 0). OK
    #              std 180 -> (180-90+360)%360 = 90 (Accumulating is at 90). OK
    #              std 270 -> (270-90+360)%360 = 180 (Planning is at 180). OK
    plotly_mapped_angle = (standard_angle_0_360 - 90 + 360) % 360

    daily_metrics_wb_rb.append({
        'Date': current_date,
        'WB': daily_wb_val,
        'RB': daily_rb_val,
        'Norm (0-1)': norm_normalized,
        'Angle (degrees)': plotly_mapped_angle # Use the transformed angle
    })

df_daily_wb_rb_evolution = pd.DataFrame(daily_metrics_wb_rb)
df_daily_wb_rb_evolution['Date'] = pd.to_datetime(df_daily_wb_rb_evolution['Date'])

# Create subplots with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for WB, RB, Norm (primary y-axis)
fig.add_trace(go.Scatter(
    x=df_daily_wb_rb_evolution['Date'],
    y=df_daily_wb_rb_evolution['WB'],
    mode='lines',
    name='WB (Workload Balance)',
    line=dict(shape='spline'),
    hovertemplate='<b>Date:</b> %{x}<br><b>WB:</b> %{y:.2f}<extra></extra>'
), secondary_y=False)

fig.add_trace(go.Scatter(
    x=df_daily_wb_rb_evolution['Date'],
    y=df_daily_wb_rb_evolution['RB'],
    mode='lines',
    name='RB (Resolution Balance)',
    line=dict(shape='spline'),
    hovertemplate='<b>Date:</b> %{x}<br><b>RB:</b> %{y:.2f}<extra></extra>'
), secondary_y=False)

fig.add_trace(go.Scatter(
    x=df_daily_wb_rb_evolution['Date'],
    y=df_daily_wb_rb_evolution['Norm (0-1)'],
    mode='lines',
    name='Magnitude (Norm normalized)',
    line=dict(shape='spline'),
    hovertemplate='<b>Date:</b> %{x}<br><b>Magnitude:</b> %{y:.2f}<extra></extra>'
), secondary_y=False)

# Add trace for Angle (secondary y-axis)
fig.add_trace(go.Scatter(
    x=df_daily_wb_rb_evolution['Date'],
    y=df_daily_wb_rb_evolution['Angle (degrees)'],
    mode='lines',
    name='Angle (degrees)',
    line=dict(shape='spline', dash='dot', color='purple'),
    hovertemplate='<b>Date:</b> %{x}<br><b>Angle:</b> %{y:.1f} degrees<extra></extra>'
), secondary_y=True)

# Update layout
fig.update_layout(
    title_text='Daily Evolution of WB, RB, Magnitude and Angle',
    xaxis_title='Date',
    yaxis_title='WB, RB, Magnitude (Range -1 to 1)', # Corrected title to reflect single scale range
    yaxis2_title='Angle (degrees)',
    legend_title='Metric',
    hovermode='x unified',
    height=600
)

fig.update_xaxes(dtick="D1", tickformat="%b %d") # Daily ticks, format as "Month Day"

# Adjust y-axis ranges for clarity
fig.update_yaxes(range=[-1.1, 1.1], secondary_y=False) # WB, RB, Norm range
fig.update_yaxes(range=[0, 360], secondary_y=True, dtick=45) # Angle range (0 to 360 degrees), with 45-degree ticks

fig.show()
save_fig(fig, 'wb_rb_time_evolution.html')

print("Daily WB, RB, Norm and Angle evolution chart generated.")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/wb_rb_time_evolution.html
Daily WB, RB, Norm and Angle evolution chart generated.


In [44]:
## Plot daily WB and RB evolution on a polar chart.

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go # Import go for update_traces
import numpy as np

# Ensure df_daily_wb and df_daily_rb are available from previous cells
# Merge the daily WB and RB dataframes on 'Date'
df_wb_rb_temporal = pd.merge(df_daily_wb, df_daily_rb, on='Date', how='inner')

# Add an identifier for the start and end points for better visualization
df_wb_rb_temporal['Point Type'] = 'Intermediate'
if not df_wb_rb_temporal.empty:
    df_wb_rb_temporal.loc[df_wb_rb_temporal.index[0], 'Point Type'] = 'Start'
    df_wb_rb_temporal.loc[df_wb_rb_temporal.index[-1], 'Point Type'] = 'End'

# Add a sequential number for each point
df_wb_rb_temporal['Point Number'] = range(1, len(df_wb_rb_temporal) + 1)

# --- Calculate polar coordinates (r, theta) for all points ---
# r (radius) = distance from center
df_wb_rb_temporal['r'] = np.sqrt(df_wb_rb_temporal['Daily WB']**2 + df_wb_rb_temporal['Daily RB']**2)
# theta (angle) = angle in degrees
# Adjust atan2 to place +WB at 0 degrees and +RB at 90 degrees
df_wb_rb_temporal['theta_degrees'] = np.degrees(np.arctan2(df_wb_rb_temporal['Daily RB'], df_wb_rb_temporal['Daily WB']))

# Initialize figure
fig = go.Figure()

if not df_wb_rb_temporal.empty:
    # Add the main continuous line for all points (drawn first so points are on top)
    fig.add_trace(go.Scatterpolar(
        r=df_wb_rb_temporal['r'],
        theta=df_wb_rb_temporal['theta_degrees'],
        mode='lines', # Only lines for the main path
        line=dict(shape='spline', color='blue', width=2), # Smooth blue line
        name='Daily Evolution',
        hoverinfo='none', # No hover for the line itself
        showlegend=False
    ))

    # --- Add the Global Project point ---
    # Ensure df_metrics (containing global rb_index, wb_index, r, theta_degrees) is available
    if 'df_metrics' in globals() and not df_metrics.empty:
        global_r = df_metrics['r'].iloc[0]
        global_theta_degrees = df_metrics['theta_degrees'].iloc[0]
        global_wb = df_metrics['WB'].iloc[0]
        global_rb = df_metrics['RB'].iloc[0]

        fig.add_trace(go.Scatterpolar(
            r=[global_r],
            theta=[global_theta_degrees],
            mode='markers',
            marker=dict(symbol='circle-dot', size=12, color='black', line=dict(width=2, color='white')),
            name='Project', # Changed from 'Project Global' to 'Project'
            hoverinfo='text',
            text=f"Project<br>WB: {global_wb:.2f}<br>RB: {global_rb:.2f}" # Changed text too
        ))

    # Add markers for all points with their point number as text
    fig.add_trace(go.Scatterpolar(
        r=df_wb_rb_temporal['r'],
        theta=df_wb_rb_temporal['theta_degrees'],
        mode='markers+text', # Markers and text (point numbers)
        marker=dict(symbol='circle', size=8, color='rgba(0,0,255,0.5)', line=dict(width=1, color='blue')),
        text=df_wb_rb_temporal['Point Number'], # Show point number as text
        textposition='top center',
        name='Daily Points',
        hoverinfo='text',
        texttemplate='<b>%{text}</b>', # Make point number text bold
        hovertext=df_wb_rb_temporal.apply(lambda row:
            f"Date: {row['Date'].strftime('%Y-%m-%d')}<br>" +
            f"WB: {row['Daily WB']:.2f}<br>" +
            f"RB: {row['Daily RB']:.2f}<br>" +
            f"Point: {row['Point Number']}", axis=1),
        showlegend=False # No legend for generic points, only start/end
    ))

    # Add specific trace for the 'Start' point to highlight it
    start_point = df_wb_rb_temporal[df_wb_rb_temporal['Point Type'] == 'Start']
    if not start_point.empty:
        fig.add_trace(go.Scatterpolar(
            r=start_point['r'],
            theta=start_point['theta_degrees'],
            mode='markers',
            marker=dict(symbol='circle', size=15, color='green', line=dict(width=2, color='black')),
            name='Start Point',
            hoverinfo='text',
            text=start_point.apply(lambda row:
                f"Date: {row['Date'].strftime('%Y-%m-%d')}<br>" +
                f"WB: {row['Daily WB']:.2f}<br>" +
                f"RB: {row['Daily RB']:.2f}<br>" +
                f"Point: {row['Point Number']}", axis=1)
        ))

    # Add specific trace for the 'End' point to highlight it
    end_point = df_wb_rb_temporal[df_wb_rb_temporal['Point Type'] == 'End']
    if not end_point.empty:
        fig.add_trace(go.Scatterpolar(
            r=end_point['r'],
            theta=end_point['theta_degrees'],
            mode='markers',
            marker=dict(symbol='diamond', size=15, color='red', line=dict(width=2, color='black')),
            name='End Point',
            hoverinfo='text',
            text=end_point.apply(lambda row:
                f"Date: {row['Date'].strftime('%Y-%m-%d')}<br>" +
                f"WB: {row['Daily WB']:.2f}<br>" +
                f"RB: {row['Daily RB']:.2f}<br>" +
                f"Point: {row['Point Number']}", axis=1)
        ))

# --- Update layout for polar coordinates ---
fig.update_layout(
    title_text='Project Tide Phase Diagram (WB vs RB Evolution)',
    height=700, # Reduced height
    width=800, # Reduced width
    showlegend=True,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02, # Position above the plot
        xanchor='right',
        x=1
    ),
    polar=dict(
        domain=dict(x=[0.1, 0.9], y=[0.1, 0.9]), # Shrink the polar plot itself to create more empty space
        radialaxis=dict(
            range=[0, np.sqrt(2) * 1.05], # Adjust range to accommodate sqrt(2) and a bit more for padding
            dtick=0.2 # Set tick interval to 0.2
        ),
        angularaxis=dict(
            rotation=90, # To align 0 degrees with 'Up' (+WB)
            direction='clockwise', # For clockwise angle increase
            tickvals=[0, 90, 180, 270],
            ticktext=['Clearing (+WB)', 'Implementing (+RB)', 'Accumulating (-WB)', 'Planning (-RB)']
        )
    )
)

# Removed x/y axis specific updates as they are now handled by polar layout

fig.show()

save_fig(fig, 'wb_rb_temporal_evolution.html')

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/wb_rb_temporal_evolution.html


## PR completion-time in the period

In [45]:
# cellname : pr-close-time

from datetime import datetime, timezone
import pandas as pd
import plotly.express as px

# Ensure selected_timedelta and repo are available from previous cells
end_date = datetime.now(timezone.utc)
start_date = end_date - selected_timedelta

pr_close_times = []

# Initialize counters for the new categories
opened_before_closed_in_period_count = 0
opened_and_closed_in_period_count = 0
opened_in_period_still_open_count = 0
opened_before_still_open_count = 0

print(f"Analyzing PRs relative to the period: {start_date.strftime('%Y-%m-%d %H:%M:%S')} to {end_date.strftime('%Y-%m-%d %H:%M:%S')}...")

# Fetch ALL pull requests (both open and closed) for comprehensive analysis
all_pulls = repo.get_pulls(state='all')

for pull in all_pulls:
    # Condition for PRs closed within the period
    if pull.closed_at and start_date <= pull.closed_at <= end_date:
        time_to_close = pull.closed_at - pull.created_at
        pr_data = {
            'number': pull.number,
            'title': pull.title,
            'creator': pull.user.login if pull.user else 'N/A',
            'created_at': pull.created_at,
            'closed_at': pull.closed_at,
            'time_to_close_days': time_to_close.total_seconds() / (24 * 3600)
        }

        # Category: Opened before period and closed in period
        if pull.created_at < start_date:
            opened_before_closed_in_period_count += 1
            pr_data['Category'] = 'Opened before period & closed in period'
            pr_close_times.append(pr_data)
        # Category: Opened and closed in period
        elif start_date <= pull.created_at <= end_date:
            opened_and_closed_in_period_count += 1
            pr_data['Category'] = 'Opened and closed in period'
            pr_close_times.append(pr_data)

    # Category: Opened in period and not yet closed
    elif start_date <= pull.created_at <= end_date and pull.closed_at is None:
        opened_in_period_still_open_count += 1

    # Category: Opened before period and still open
    elif pull.created_at < start_date and pull.closed_at is None:
        opened_before_still_open_count += 1

print("\n--- PR Categorization for the Period ---")
print(f"* PRs opened before period and closed in period: {opened_before_closed_in_period_count}")
print(f"* PRs opened and closed in period: {opened_and_closed_in_period_count}")
print(f"* PRs opened in period and not closed: {opened_in_period_still_open_count}")
print(f"* PRs opened before period and still open: {opened_before_still_open_count}")

df_pr_close_times = pd.DataFrame(pr_close_times)

if not df_pr_close_times.empty:
    print(f"\nFound {len(df_pr_close_times)} PRs closed within the period for detailed analysis.")
    display(df_pr_close_times.head())

    print("\n--- Summary of PR Close Times (Closed in Period) ---")
    print(f"Average time to close PR: {df_pr_close_times['time_to_close_days'].mean():.2f} days")
    print(f"Median time to close PR: {df_pr_close_times['time_to_close_days'].median():.2f} days")
    print(f"Minimum time to close PR: {df_pr_close_times['time_to_close_days'].min():.2f} days")
    print(f"Maximum time to close PR: {df_pr_close_times['time_to_close_days'].max():.2f} days")

    # Calculate nbins to ensure 1-day intervals
    max_days = df_pr_close_times['time_to_close_days'].max()
    nbins_calculated = int(max_days) + 1 if max_days > 0 else 1 # Ensure at least 1 bin if data exists

    # Plotting a histogram of close times with color distinction
    fig = px.histogram(df_pr_close_times,
                       x='time_to_close_days',
                       nbins=nbins_calculated, # Use calculated nbins for 1-day intervals
                       color='Category', # Use the new Category column for color
                       barmode='stack', # Changed to stack mode
                       title='PR completion-time histogram by Category',
                       labels={'time_to_close_days': 'Time to Close (Days)', 'count': 'Number of PRs'})
    fig.update_traces(opacity=0.75) # Adjust opacity for better visibility of overlaid bars
    fig.update_layout(xaxis=dict(dtick=1)) # Set x-axis ticks to 1-unit intervals
    fig.show()
    save_fig(fig, 'pr_close_times_histogram.html')
else:
    print("\nNo PRs were closed within the selected period for detailed time analysis.")

Analyzing PRs relative to the period: 2025-11-02 14:19:00 to 2025-11-30 14:19:00...

--- PR Categorization for the Period ---
* PRs opened before period and closed in period: 33
* PRs opened and closed in period: 60
* PRs opened in period and not closed: 2
* PRs opened before period and still open: 0

Found 93 PRs closed within the period for detailed analysis.


,number,title,creator,created_at,closed_at,time_to_close_days,Category
0,429,Feat/403/updating manual,OGabiruuu,2025-11-29 14:40:05+00:00,2025-11-29 15:08:39+00:00,0.019838,Opened and closed in period
1,428,bug: fix game modes menu layout,LeticiaBN,2025-11-29 14:32:45+00:00,2025-11-29 15:30:07+00:00,0.039838,Opened and closed in period
2,426,refactor: add game_over_service,otaviobia,2025-11-28 00:39:31+00:00,2025-11-28 23:23:25+00:00,0.947153,Opened and closed in period
3,424,[BUG] Fix Autoplay Mode Freeze After Wait-for-...,DeguShi,2025-11-26 22:27:00+00:00,2025-11-27 21:06:45+00:00,0.944271,Opened and closed in period
4,423,[FEAT] rainbow snake,trqt,2025-11-26 15:12:42+00:00,2025-11-26 20:47:29+00:00,0.232488,Opened and closed in period



--- Summary of PR Close Times (Closed in Period) ---
Average time to close PR: 11.82 days
Median time to close PR: 6.91 days
Minimum time to close PR: 0.00 days
Maximum time to close PR: 36.37 days


Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/pr_close_times_histogram.html


In [46]:
## Visualize PR completion times with IQR by category.

import plotly.graph_objects as go
import pandas as pd
import plotly.express as px # Import plotly.express to use its color sequences if needed

# Ensure the df_pr_close_times DataFrame is not empty
if not df_pr_close_times.empty:
    # Define colors for categories (using plotly express default sequence for consistency)
    colors = px.colors.qualitative.Plotly # Or choose specific colors: ['blue', 'orange']
    color_map = {
        'Opened and closed in period': colors[0], # e.g., blue
        'Opened before period & closed in period': colors[1] # e.g., orange
    }

    # Create the figure
    fig = go.Figure()

    # Calculate quartiles for each category
    category_quartiles = df_pr_close_times.groupby('Category')['time_to_close_days'].quantile([0.25, 0.5, 0.75]).unstack()
    category_quartiles.columns = ['Q1', 'Median', 'Q3']

    # Iterate over unique categories and create a trace for each
    for i, (category_name, color_val) in enumerate(color_map.items()):
        df_category = df_pr_close_times[df_pr_close_times['Category'] == category_name]
        if not df_category.empty:
            fig.add_trace(go.Scatter(
                x=df_category['number'],
                y=df_category['time_to_close_days'],
                mode='markers',
                marker=dict(
                    size=10,
                    opacity=0.8,
                    line=dict(width=1, color='DarkSlateGrey'),
                    color=color_val # Set color explicitly for this trace
                ),
                name=category_name, # Set the name for the legend entry
                hoverinfo='text',
                # Custom hover text to display detailed PR information
                text=df_category.apply(lambda row:
                    f"PR #{row['number']}<br>" +
                    f"Title: {row['title']}<br>" +
                    f"Creator: {row['creator']}<br>" +
                    f"Created: {row['created_at'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
                    f"Closed: {row['closed_at'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
                    f"Time to Close: {row['time_to_close_days']:.2f} days<br>" +
                    f"Category: {row['Category']}", axis=1)
            ))

            # Add a transparent rectangle for IQR (Q1 to Q3) for this category
            if category_name in category_quartiles.index:
                q1 = category_quartiles.loc[category_name, 'Q1']
                q3 = category_quartiles.loc[category_name, 'Q3']
                median = category_quartiles.loc[category_name, 'Median']

                fig.add_shape(type='rect',
                              xref='paper', yref='y',
                              x0=0, x1=0.75, # Spans the constrained plot area width
                              y0=q1, y1=q3,
                              fillcolor=color_val,
                              opacity=0.15, # Transparent fill
                              layer='below',
                              line_width=0)

                # Add median line for this category
                fig.add_hline(y=median,
                              line_dash='dash',
                              line_color=color_val,
                              line_width=1)

                # Define x-position for annotations
                x_pos_base = 0.75 + 0.01 # Start just outside the new x-axis domain

                # Annotation for Median
                fig.add_annotation(
                    xref='paper', yref='y',
                    x=x_pos_base, y=median,
                    text=f"Median: {median:.2f} days",
                    showarrow=False,
                    font=dict(color=color_val, size=12), # Increased font size to 12
                    xanchor='left', yanchor='middle'
                )


    # Update layout for title and axis labels
    fig.update_layout(
        title='PR completion time with IQR by Category',
        xaxis_title='Pull Request Number',
        yaxis_title='Time to Close (Days)',
        height=600,
        width=1000, # Increased canvas width
        hovermode='closest',
        showlegend=True, # Show legend to differentiate categories
        legend_title='PR Category',
        legend=dict(
            orientation='h', # Horizontal orientation for bottom placement
            yanchor='bottom',   # Anchor to the bottom
            y=-0.3,             # Position further below the plot area
            xanchor='center',
            x=0.5              # Center horizontally
        ),
        xaxis=dict(
            domain=[0, 0.75] # Constrain x-axis to 75% of the figure width
        )
    )

    # --- NEW: Add annotations for Q1, Median, Q3 below the legend ---
    # Adjust height to accommodate these new annotations
    fig.update_layout(height=800) # Increase figure height here

    # Starting Y position for the first annotation block, below the legend (-0.3)
    annotation_start_y_pos = -0.45 # Relative to paper coordinates

    for i, (category_name, color_val) in enumerate(color_map.items()):
        if category_name in category_quartiles.index:
            q1 = category_quartiles.loc[category_name, 'Q1']
            median = category_quartiles.loc[category_name, 'Median']
            q3 = category_quartiles.loc[category_name, 'Q3']

            # Format the text for the annotation
            # Using HTML for bold category name
            annotation_text = f"<b>{category_name}</b><br>" \
                              f"Q1: {q1:.2f} days<br>" \
                              f"Median: {median:.2f} days<br>" \
                              f"Q3: {q3:.2f} days"

            fig.add_annotation(
                xref='paper', yref='paper',
                x=0.5, # Center horizontally below the legend
                y=annotation_start_y_pos - (0.15 * i), # Stack vertically, adjust 0.15 for spacing between blocks
                text=annotation_text,
                showarrow=False,
                font=dict(color=color_val, size=10),
                xanchor='center', yanchor='top' # Anchor the top of the text block to the y position
            )

    # Display the plot
    fig.show()
    save_fig(fig, 'pr_close_time_scatter_plot.html')

    # Print quartile values for each category
    print("\n--- Quartile Values per PR Category ---")
    for category_name in category_quartiles.index:
        q1_val = category_quartiles.loc[category_name, 'Q1']
        median_val = category_quartiles.loc[category_name, 'Median']
        q3_val = category_quartiles.loc[category_name, 'Q3']
        print(f"  Category: {category_name}")
        print(f"    Q1: {q1_val:.2f} days")
        print(f"    Median: {median_val:.2f} days")
        print(f"    Q3: {q3_val:.2f} days")

else:
    print("No PRs were opened and closed within the selected period to plot.")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/pr_close_time_scatter_plot.html

--- Quartile Values per PR Category ---
  Category: Opened and closed in period
    Q1: 0.14 days
    Median: 0.89 days
    Q3: 6.04 days
  Category: Opened before period & closed in period
    Q1: 27.96 days
    Median: 28.20 days
    Q3: 30.12 days


In [47]:
## Visualize PR status distribution with a pie chart.

import plotly.express as px
import pandas as pd

# Data from the previous cell's execution
pr_category_data = {
    'Category': [
        'Opened before period & closed in period',
        'Opened & closed in period',
        'Opened in period & not closed',
        'Opened before period & still open'
    ],
    'Count': [
        opened_before_closed_in_period_count,
        opened_and_closed_in_period_count,
        opened_in_period_still_open_count,
        opened_before_still_open_count
    ]
}

df_pr_categories = pd.DataFrame(pr_category_data)

# Filter out categories with 0 count to avoid empty slices
df_pr_categories = df_pr_categories[df_pr_categories['Count'] > 0]

if df_pr_categories.empty:
    print("No PR activity found to categorize for the pie chart.")
else:
    fig = px.pie(
        df_pr_categories,
        values='Count',
        names='Category', # Use the raw category name for slice identification
        title='Distribution of PR status',
        hole=0.3, # Optional: creates a donut chart
        color_discrete_sequence=px.colors.qualitative.Pastel # Use a pastel color sequence
    )

    fig.update_traces(
        textposition='outside',
        textinfo='percent+label+value',
        textfont_color='black'
        # Optional: rotate labels if they overlap
        # insidetextorientation='radial'
    )
    fig.update_layout(
        showlegend=True,
        legend=dict(
            orientation='h', # Horizontal orientation for bottom placement
            yanchor='bottom',   # Anchor to the bottom
            y=-0.3,             # Position further below the plot area
            xanchor='center',
            x=0.5              # Center horizontally
        )
    )
    fig.show()
    save_fig(fig, 'pr_categories_pie_chart.html')

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/pr_categories_pie_chart.html


# DEVELOPERS OVERVIEW

## Engagement

### Engament chart

In [48]:
# cellname : engagement-plot

import plotly.express as px
import pandas as pd

# Calculate the 'Discussion' metric
df_plot_discussion = df_people.copy()
df_plot_discussion['Discussion'] = df_plot_discussion['issues_opened_count'] + df_plot_discussion['issue_comment_event_count']

# Filter out developers who have no activity in either PRs Opened or Discussion
df_plot_discussion = df_plot_discussion[
    (df_plot_discussion['pull_requests_opened_count'] > 0) |
    (df_plot_discussion['Discussion'] > 0)
].copy()

# --- Aggregate data for unique (PRs Opened, Discussion) coordinates ---
aggregated_df = df_plot_discussion.groupby(['pull_requests_opened_count', 'Discussion']).agg(
    github_logins=('github_login', lambda x: ', '.join(x)),
    names=('name', lambda x: ', '.join(x)),
    num_devs=('github_login', 'count')
).reset_index()

# Create a unique key for each aggregated point for coloring with distinct discrete colors
# This key will also be used initially as the legend name before being replaced by logins
aggregated_df['point_category'] = 'PRs ' + aggregated_df['pull_requests_opened_count'].astype(str) + ', Discussion ' + aggregated_df['Discussion'].astype(str)

# Add a new column to aggregated_df to determine marker style
aggregated_df['marker_type'] = aggregated_df['num_devs'].apply(lambda x: 'Single Contributor' if x == 1 else 'Multiple Contributors')

# Define a custom color sequence using only Dark24
custom_color_sequence = px.colors.qualitative.Dark24

# Create the scatter plot with aggregated data
fig = px.scatter(
    aggregated_df,
    x='pull_requests_opened_count',
    y='Discussion',
    size='num_devs', # Size of marker based on number of developers at this point
    color='point_category', # Use the categorical key for distinct colors
    color_discrete_sequence=custom_color_sequence, # Use the custom, extended palette
    symbol='marker_type', # Use the new column to determine marker symbol (solid vs. outlined)
    symbol_map={'Single Contributor': 'circle', 'Multiple Contributors': 'circle-open'}, # Explicitly map styles
    text=None, # Removed text labels from directly on the bubbles
    title='Engagement chart',
    labels={
        'pull_requests_opened_count': 'PRs Opened',
        'Discussion': 'Discussion (Issues Opened + Comments)',
        'num_devs': 'Number of Contributors',
        'github_logins': 'Contributors',
        'point_category': 'Contributors' # This will be the initial legend title, replaced later
    },
    hover_name='names', # Show aggregated names on hover
    hover_data={
        'github_logins': True, # Also show aggregated logins on hover
        'num_devs': True,
        'pull_requests_opened_count': True,
        'Discussion': True,
        'names': False,
        'point_category': False
    }
)

# The update_traces for textposition will now do nothing as text=None
# fig.update_traces(textposition='middle right', textfont_size=10)

# Increase border thickness for 'circle-open' symbols, inheriting color
fig.update_traces(marker=dict(line=dict(width=8, color='black')), selector=dict(symbol='circle-open'))

# Create a mapping from point_category to github_logins for legend renaming
category_to_logins_map = aggregated_df.set_index('point_category')['github_logins'].to_dict()

# Function to extract the base point_category from the full trace name
def get_base_point_category(full_trace_name):
    # The trace name will be 'point_category_value, marker_type_value' if both color and symbol are used
    # We want to split at the last comma to get 'point_category_value'
    parts = full_trace_name.rsplit(', ', 1)
    if len(parts) > 1 and (parts[-1] == 'Single Contributor' or parts[-1] == 'Multiple Contributors'):
        return parts[0]
    return full_trace_name # Fallback if name doesn't match expected pattern (e.g., if only color is used)

# Update the legend entry names to show github_logins
fig.for_each_trace(lambda trace: trace.update(name=category_to_logins_map[get_base_point_category(trace.name)]))

fig.update_layout(
    height=1200, # Increased height to accommodate the legend
    xaxis_title='PRs Opened',
    yaxis_title='Discussion (Issues Opened + Comments)',
    legend_title='Contributors',
    legend=dict(
        orientation='v', # Vertical orientation
        yanchor='top', # Anchor to the top
        y=-0.2, # Position below the chart (adjust as needed)
        xanchor='left',
        x=0
    )
)

fig.show()

save_fig(fig, 'engagement_plot.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/engagement_plot.html


### Engagement magnitude (EM) index

In [49]:
# cellname :em-agregado
import numpy as np
import pandas as pd

# --- 1. Calculate Project-wide totals for Discussion and PRs Opened ---
# Summing across all individuals in df_people
total_project_discussion = df_people['issues_opened_count'].sum() + df_people['issue_comment_event_count'].sum()
total_project_prs_opened = df_people['pull_requests_opened_count'].sum()

print(f"Total Project Discussion (Issues Opened + Comments): {total_project_discussion}")
print(f"Total Project PRs Opened: {total_project_prs_opened}")

# --- 2. Retrieve max individual values for normalization ---
# These values (max_discussion, max_prs_opened) refer to the highest individual contribution.
# Calculate them from df_people
df_people_discussion_temp = df_people['issues_opened_count'] + df_people['issue_comment_event_count']
max_discussion = df_people_discussion_temp.max()
max_prs_opened = df_people['pull_requests_opened_count'].max()

# Handle division by zero if no activity of a certain type
if max_discussion == 0: max_discussion = 1
if max_prs_opened == 0: max_prs_opened = 1

# --- 3. Calculate count of active contributors for proper scaling ---
# An 'active' contributor for EM is one with >0 discussion or >0 PRs opened
active_contributors_for_em_count = df_people[
    (df_people_discussion_temp > 0) | (df_people['pull_requests_opened_count'] > 0)
].shape[0]

# Ensure at least one active contributor to avoid division by zero later
if active_contributors_for_em_count == 0: active_contributors_for_em_count = 1


# --- 4. Normalize Project totals using individual maximums ---
# These are 'project totals relative to the best individual performance'
normalized_project_discussion = total_project_discussion / max_discussion
normalized_project_prs_opened = total_project_prs_opened / max_prs_opened

print(f"Normalized Project Discussion (relative to max individual): {normalized_project_discussion:.2f}")
print(f"Normalized Project PRs Opened (relative to max individual): {normalized_project_prs_opened:.2f}")

# --- 5. Calculate Project EM (raw magnitude) ---
project_em_raw = np.sqrt(normalized_project_discussion**2 + normalized_project_prs_opened**2)

# --- 6. Scale Project EM to be between 0 and 1 ---
# The maximum possible value for project_em_raw, if all 'active_contributors_for_em_count'
# were performing at 'max_discussion' and 'max_prs_opened' level, would be
# active_contributors_for_em_count * sqrt(max_discussion^2/max_discussion^2 + max_prs_opened^2/max_prs_opened^2)
# = active_contributors_for_em_count * sqrt(1+1) = active_contributors_for_em_count * sqrt(2)
max_theoretical_project_em_raw = active_contributors_for_em_count * np.sqrt(2)

if max_theoretical_project_em_raw > 0:
    project_em = project_em_raw / max_theoretical_project_em_raw
else:
    project_em = 0.0 # If no activity, project EM is 0

print(f"\nProject Engagement Magnitude (EM) (0-1 scaled): {project_em:.2f}")

Total Project Discussion (Issues Opened + Comments): 338
Total Project PRs Opened: 62
Normalized Project Discussion (relative to max individual): 10.56
Normalized Project PRs Opened (relative to max individual): 12.40

Project Engagement Magnitude (EM) (0-1 scaled): 0.26


In [50]:
# cellname : engagement-index

import plotly.express as px
import pandas as pd
import numpy as np

# Calculate the 'Discussion' metric if not already present
df_bci = df_people.copy()
df_bci['Discussion'] = df_bci['issues_opened_count'] + df_bci['issue_comment_event_count']

# Filter out developers with BCI equal to 0 (no contributions in these categories)
df_em_plot = df_bci[((df_bci['Discussion'] > 0) | (df_bci['pull_requests_opened_count'] > 0))].copy()

# --- Normalize coordinates (Discussion, PRs Opened) between 0 and 1 ---
max_discussion = df_em_plot['Discussion'].max()
max_prs_opened = df_em_plot['pull_requests_opened_count'].max()

df_em_plot['Normalized_Discussion'] = 0
if max_discussion > 0:
    df_em_plot['Normalized_Discussion'] = (df_em_plot['Discussion'] / max_discussion)

df_em_plot['Normalized_PRs_Opened'] = 0
if max_prs_opened > 0:
    df_em_plot['Normalized_PRs_Opened'] = (df_em_plot['pull_requests_opened_count'] / max_prs_opened)

# Calculate the Engagement Magnitude Index (EM) using normalized values
# EM = sqrt(Normalized_Discussion^2 + Normalized_PRs_Opened^2)
df_em_plot['EM'] = np.sqrt(df_em_plot['Normalized_Discussion']**2 + df_em_plot['Normalized_PRs_Opened']**2)

# Normalize EM by its theoretical maximum of sqrt(2) to scale it between 0 and 1
if not df_em_plot['EM'].empty:
    df_em_plot['EM'] = df_em_plot['EM'] / np.sqrt(2)

# Sort the DataFrame by EM in descending order
df_em_plot = df_em_plot.sort_values(by='EM', ascending=False)

# Calculate the average EM for all contributors
average_em = df_em_plot['EM'].mean()

# Ensure project_em is available from the 'em-agregado' cell (H27fmKkNGTum)
# Fallback if it hasn't been run or is not defined
if 'project_em' not in globals():
    # Re-calculate project_em if not available (should ideally come from H27fmKkNGTum)
    total_project_discussion = df_people['issues_opened_count'].sum() + df_people['issue_comment_event_count'].sum()
    total_project_prs_opened = df_people['pull_requests_opened_count'].sum()

    max_discussion_fallback = df_em_plot['Discussion'].max() if df_em_plot['Discussion'].max() > 0 else 1
    max_prs_opened_fallback = df_em_plot['pull_requests_opened_count'].max() if df_em_plot['pull_requests_opened_count'].max() > 0 else 1

    normalized_project_discussion_fallback = total_project_discussion / max_discussion_fallback
    normalized_project_prs_opened_fallback = total_project_prs_opened / max_prs_opened_fallback

    project_em_raw_fallback = np.sqrt(normalized_project_discussion_fallback**2 + normalized_project_prs_opened_fallback**2)
    project_em = project_em_raw_fallback / np.sqrt(2)

# Create the bar chart for EM
fig = px.bar(
    df_em_plot,
    x='github_login',
    y='EM',
    title='Engagement magnitude (EM) index',
    labels={
        'github_login': 'Contributor',
        'EM': 'EM'
    },
    hover_name='name',
    hover_data={
        'github_login': False,
        'name': True,
        'Discussion': True,
        'pull_requests_opened_count': True,
        'Normalized_Discussion': ':.2f',
        'Normalized_PRs_Opened': ':.2f',
        'EM': ':.2f' # Format EM to 2 decimal places in hover
    },
    text_auto='.1f' # Format EM to 1 decimal place on top of bars
)

# Adjust x-axis to show labels vertically
fig.update_xaxes(tickangle=90, tickfont=dict(size=10))
fig.update_yaxes(rangemode='tozero') # Ensure y-axis starts at zero

# Optional: adjust text position if needed (default 'auto' for bar usually puts it on top)
fig.update_traces(textposition='outside')

# Removed horizontal line for the average EM of all contributors as requested
# fig.add_hline(y=average_em, line_width=1, line_dash="dash", line_color="gray",
#               annotation_text=f"Average EM: {average_em:.2f}",
#               annotation_position="bottom right",
#               annotation_font_color="gray")

# Add horizontal line for the aggregated project EM
fig.add_hline(y=project_em, line_width=1, line_dash="dash", line_color="black",
              annotation_text=f"Project EM: {project_em:.2f}",
              annotation_position="top right",
              annotation_font_color="black")

fig.show()
save_fig(fig, 'em_index.html')

print("Figure saved to: /content/gdrive/My Drive/naja-2025-11-27/em_index.html")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/em_index.html
Figure saved to: /content/gdrive/My Drive/naja-2025-11-27/em_index.html


REMI is the norm of the two-dimensional contribution vector whose coordinates are the developer’s Discussion score and the number of PRs opened. In practice, EMI captures the overall intensity of a developer’s participatory and initiatory actions by combining conversational engagement with concrete technical initiation into a single magnitude.

## Delivery

### Delivery chart

In [51]:
# cellname : dm-agregado
import numpy as np
import pandas as pd

# --- 1. Calculate Project-wide totals for Code Changes and PRs Merged ---
# Summing across all individuals in df_people
total_project_code_changes = df_people['total_additions'].sum() + df_people['total_deletions'].sum()
total_project_prs_merged = df_people['pull_requests_merged_count'].sum()

print(f"Total Project Code Changes (Additions + Deletions): {total_project_code_changes}")
print(f"Total Project PRs Merged: {total_project_prs_merged}")

# --- 2. Retrieve max individual values for normalization ---
# These values (max_code_changes, max_prs_merged) refer to the highest individual contribution.
# Calculate them from df_people
df_people_code_changes_temp = df_people['total_additions'] + df_people['total_deletions']
max_code_changes = df_people_code_changes_temp.max()
max_prs_merged = df_people['pull_requests_merged_count'].max()

# Handle division by zero if no activity of a certain type
if max_code_changes == 0: max_code_changes = 1
if max_prs_merged == 0: max_prs_merged = 1

# --- 3. Calculate count of active contributors for proper scaling ---
# An 'active' contributor for DM is one with >0 code changes or >0 PRs merged
active_contributors_for_dm_count = df_people[
    (df_people_code_changes_temp > 0) | (df_people['pull_requests_merged_count'] > 0)
].shape[0]

# Ensure at least one active contributor to avoid division by zero later
if active_contributors_for_dm_count == 0: active_contributors_for_dm_count = 1

# --- 4. Normalize Project totals using individual maximums ---
normalized_project_code_changes = total_project_code_changes / max_code_changes
normalized_project_prs_merged = total_project_prs_merged / max_prs_merged

print(f"Normalized Project Code Changes (relative to max individual): {normalized_project_code_changes:.2f}")
print(f"Normalized Project PRs Merged (relative to max individual): {normalized_project_prs_merged:.2f}")

# --- 5. Calculate Project DM (raw magnitude) ---
project_dm_raw = np.sqrt(normalized_project_code_changes**2 + normalized_project_prs_merged**2)

# --- 6. Scale Project DM to be between 0 and 1 ---
# The maximum possible value for project_dm_raw, if all 'active_contributors_for_dm_count'
# were performing at 'max_code_changes' and 'max_prs_merged' level, would be
# active_contributors_for_dm_count * sqrt(max_code_changes^2/max_code_changes^2 + max_prs_merged^2/max_prs_merged^2)
# = active_contributors_for_dm_count * sqrt(1+1) = active_contributors_for_dm_count * sqrt(2)
max_theoretical_project_dm_raw = active_contributors_for_dm_count * np.sqrt(2)

if max_theoretical_project_dm_raw > 0:
    project_dm = project_dm_raw / max_theoretical_project_dm_raw
else:
    project_dm = 0.0 # If no activity, project DM is 0

print(f"\nProject Delivery Magnitude (DM) (0-1 scaled): {project_dm:.2f}")

Total Project Code Changes (Additions + Deletions): 26399
Total Project PRs Merged: 39
Normalized Project Code Changes (relative to max individual): 2.82
Normalized Project PRs Merged (relative to max individual): 5.57

Project Delivery Magnitude (DM) (0-1 scaled): 0.18


In [52]:
# cellname : deliery-plot

import plotly.express as px
import pandas as pd

# Calculate the 'Code Changes' metric
df_plot_code_changes = df_people.copy()
df_plot_code_changes['Code Changes'] = df_plot_code_changes['total_additions'] + df_plot_code_changes['total_deletions']

# Filter out developers who have no activity in either PRs Merged or Code Changes
df_plot_code_changes = df_plot_code_changes[
    (df_plot_code_changes['pull_requests_merged_count'] > 0) |
    (df_plot_code_changes['Code Changes'] > 0)
].copy()

# --- Aggregate data for unique (PRs merged, Code Changes) coordinates ---
aggregated_df = df_plot_code_changes.groupby(['pull_requests_merged_count', 'Code Changes']).agg(
    github_logins=('github_login', lambda x: ', '.join(x)),
    names=('name', lambda x: ', '.join(x)),
    num_devs=('github_login', 'count')
).reset_index()

# Create a unique key for each aggregated point for coloring with distinct discrete colors
aggregated_df['point_category'] = 'PRs Merged ' + aggregated_df['pull_requests_merged_count'].astype(str) + ', Changes ' + aggregated_df['Code Changes'].astype(str)

# Add a new column to aggregated_df to determine marker style
# Updated: 'circle' for single, 'circle-open' for multiple. 'circle-dot' would be third if needed.
aggregated_df['marker_type'] = aggregated_df['num_devs'].apply(lambda x: 'Single Contributor' if x == 1 else 'Multiple Contributors')

# Define a custom color sequence using only Dark24
custom_color_sequence = px.colors.qualitative.Dark24

# Create the scatter plot with aggregated data (axes swapped)
fig = px.scatter(
    aggregated_df,
    x='Code Changes', # Swapped axis
    y='pull_requests_merged_count', # Swapped axis
    size='num_devs', # Size of marker based on number of developers at this point
    color='point_category', # Use the categorical key for distinct colors
    color_discrete_sequence=custom_color_sequence, # Use the custom, extended palette
    symbol='marker_type', # Use the new column to determine marker symbol (solid vs. outlined)
    symbol_map={'Single Contributor': 'circle', 'Multiple Contributors': 'circle-open'}, # Updated symbol_map
    text=None, # Removed text labels from directly on the bubbles
    title='Delivery chart',
    labels={
        'Code Changes': 'Code Changes (Additions + Deletions)', # Updated label
        'pull_requests_merged_count': 'PRs Merged', # Updated label
        'num_devs': 'Number of Contributors',
        'github_logins': 'Contributors',
        'point_category': 'Contributors' # This will be the initial legend title, replaced later
    },
    hover_name='names', # Show aggregated names on hover
    hover_data={
        'github_logins': True, # Also show aggregated logins on hover
        'num_devs': True,
        'pull_requests_merged_count': True,
        'Code Changes': True,
        'names': False, # Don't duplicate hover_name in hover_data
        'point_category': False # Don't show this in hover data, as labels are clear
    }
)

# Apply marker styling to ensure proportionality with small base size
# sizeref: smaller value makes markers larger. Larger value makes markers smaller.
# Let's target a max size of ~10 pixels for the largest 'num_devs' (reduced from 20)
max_num_devs_val = aggregated_df['num_devs'].max()
if max_num_devs_val > 0:
    target_max_size = 10 # pixels for the largest bubble (reduced)
    sizeref_val = max_num_devs_val / target_max_size
else:
    sizeref_val = 1 # avoid division by zero if all num_devs are 0 or 1

fig.update_traces(
    marker=dict(
        sizemode='diameter', # Scale by diameter
        sizeref=sizeref_val, # Reference value for scaling
        sizemin=2,           # Minimum size in pixels for the smallest 'num_devs' (reduced from 4)
        line=dict(width=1, color='DarkSlateGrey') # Default line for all
    ),
    selector=dict(mode='markers') # Apply to all scatter markers
)

# If specific line widths for symbols are desired, they can be added here
# For example, to make the 'circle-open' slightly thicker outline
fig.update_traces(marker=dict(line=dict(width=2)), selector=dict(symbol='circle-open'))


# Create a mapping from point_category to github_logins for legend renaming
category_to_logins_map = aggregated_df.set_index('point_category')['github_logins'].to_dict()

# Function to extract the base point_category from the full trace name
def get_base_point_category(full_trace_name):
    # The trace name will be 'point_category_value, marker_type_value' if both color and symbol are used
    # We want to split at the last comma to get 'point_category_value'
    parts = full_trace_name.rsplit(', ', 1)
    if len(parts) > 1 and (parts[-1] == 'Single Contributor' or parts[-1] == 'Multiple Contributors'):
        return parts[0]
    return full_trace_name # Fallback if name doesn't match expected pattern (e.g., if only color is used)

# Update the legend entry names to show github_logins
fig.for_each_trace(lambda trace: trace.update(name=category_to_logins_map[get_base_point_category(trace.name)]))

fig.update_layout(
    height=800, # Reduced height
    xaxis_title='Code Changes (Additions + Deletions)', # Swapped axis title
    yaxis_title='PRs Merged',
    legend_title='Contributors',
    legend=dict(
        orientation='v', # Vertical orientation
        yanchor='top', # Anchor to the top
        y=-0.2, # Position below the chart (adjust as needed)
        xanchor='left',
        x=0
    )
)

fig.show()

save_fig(fig, 'delivery_plot.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/delivery_plot.html


### Delivery magnitude (DM) index




In [53]:
# cellname : delivery-index
import plotly.express as px
import pandas as pd
import numpy as np

# Calculate the 'Code Changes' metric if not already present
df_dm = df_people.copy()
df_dm['Code Changes'] = df_dm['total_additions'] + df_dm['total_deletions']

# Filter out developers with no contributions in these categories
df_dm_plot = df_dm[((df_dm['Code Changes'] > 0) | (df_dm['pull_requests_merged_count'] > 0))].copy()

# --- Normalize coordinates (Code Changes, PRs Merged) between 0 and 1 ---
max_code_changes = df_dm_plot['Code Changes'].max()
max_prs_merged = df_dm_plot['pull_requests_merged_count'].max()

df_dm_plot['Normalized_Code_Changes'] = 0
if max_code_changes > 0:
    df_dm_plot['Normalized_Code_Changes'] = (df_dm_plot['Code Changes'] / max_code_changes)

df_dm_plot['Normalized_PRs_Merged'] = 0
if max_prs_merged > 0:
    df_dm_plot['Normalized_PRs_Merged'] = (df_dm_plot['pull_requests_merged_count'] / max_prs_merged)

# Calculate the Delivery Magnitude Index (DM) using normalized values
# DM = sqrt(Normalized_Code_Changes^2 + Normalized_PRs_Merged^2)
df_dm_plot['DM'] = np.sqrt(df_dm_plot['Normalized_Code_Changes']**2 + df_dm_plot['Normalized_PRs_Merged']**2)

# Normalize the final index value so that it fits in [0, 1]
# The maximum possible value for DM is sqrt(1^2 + 1^2) = sqrt(2)
if not df_dm_plot['DM'].empty:
    df_dm_plot['DM'] = df_dm_plot['DM'] / np.sqrt(2)

# Sort the DataFrame by DM in descending order
df_dm_plot = df_dm_plot.sort_values(by='DM', ascending=False)

# Ensure project_dm is available from the 'dm-agregado' cell (hxkget_HZ2JM)
# Fallback if it hasn't been run or is not defined
if 'project_dm' not in globals():
    # Re-calculate project_dm if not available (should ideally come from hxkget_HZ2JM)
    total_project_code_changes = df_people['total_additions'].sum() + df_people['total_deletions'].sum()
    total_project_prs_merged = df_people['pull_requests_merged_count'].sum()

    max_code_changes_fallback = df_dm_plot['Code Changes'].max() if df_dm_plot['Code Changes'].max() > 0 else 1
    max_prs_merged_fallback = df_dm_plot['pull_requests_merged_count'].max() if df_dm_plot['pull_requests_merged_count'].max() > 0 else 1

    normalized_project_code_changes_fallback = total_project_code_changes / max_code_changes_fallback
    normalized_project_prs_merged_fallback = total_project_prs_merged / max_prs_merged_fallback

    project_dm_raw_fallback = np.sqrt(normalized_project_code_changes_fallback**2 + normalized_project_prs_merged_fallback**2)
    active_contributors_for_dm_count = df_people[
        (df_people['total_additions'] + df_people['total_deletions'] > 0) | (df_people['pull_requests_merged_count'] > 0)
    ].shape[0]
    if active_contributors_for_dm_count == 0: active_contributors_for_dm_count = 1
    project_dm = project_dm_raw_fallback / (active_contributors_for_dm_count * np.sqrt(2))

# Create the bar chart for DM
fig = px.bar(
    df_dm_plot,
    x='github_login',
    y='DM',
    title='Delivery magnitude (DM)',
    labels={
        'github_login': 'Contributor',
        'DM': 'DM'
    },
    hover_name='name',
    hover_data={
        'github_login': False,
        'name': True,
        'Code Changes': True,
        'pull_requests_merged_count': True,
        'Normalized_Code_Changes': ':.2f',
        'Normalized_PRs_Merged': ':.2f',
        'DM': ':.2f' # Format to 2 decimal places in hover
    },
    text_auto='.1f' # Format to 1 decimal place on top of bars
)

# Adjust x-axis to show labels vertically
fig.update_xaxes(tickangle=90, tickfont=dict(size=10))
fig.update_yaxes(rangemode='tozero') # Ensure y-axis starts at zero

# Adjust text position
fig.update_traces(textposition='outside')

# Add horizontal line for the aggregated project DM
fig.add_hline(y=project_dm, line_width=1, line_dash="dash", line_color="black",
              annotation_text=f"Project DM: {project_dm:.2f}",
              annotation_position="top right",
              annotation_font_color="black")

fig.show()

save_fig(fig, 'dm_index.html')

print("Figure saved to: /content/gdrive/My Drive/naja-2025-11-28/dm_index.html")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/dm_index.html
Figure saved to: /content/gdrive/My Drive/naja-2025-11-28/dm_index.html


The Relative Delivery Magnitude Index (RDMI) is the norm of the two-dimensional delivery vector whose components are the developer’s code changes and the number of PRs merged. Conceptually, CDI represents the overall strength of a developer’s effective code delivery by combining the volume of code produced with the number of contributions successfully integrated into the project.

## Engagement-Delivery Map

In [54]:
#cellname : em-dm-chart

import plotly.express as px
import pandas as pd
import numpy as np # Ensure numpy is imported

# Ensure df_em_plot and df_dm_plot are available and contain the calculated indices
# If this cell is run independently, ensure these DFs are regenerated or loaded

# Select relevant columns from each DataFrame
df_em = df_em_plot[['github_login', 'name', 'EM']]
df_dm = df_dm_plot[['github_login', 'name', 'DM']]

# Merge the two DataFrames on 'github_login'
# Use an outer merge to include all contributors who appear in either index calculation
df_combined_indices = pd.merge(df_em, df_dm,
                               how='outer',
                               on='github_login',
                               suffixes=('_em', '_dm'))

# Handle cases where a contributor might not have activity for one of the indices
# (e.g., no PRs opened/comments for EM, or no code changes/PRs merged for DM)
df_combined_indices['EM'] = df_combined_indices['EM'].fillna(0)
df_combined_indices['DM'] = df_combined_indices['DM'].fillna(0)

# Resolve potential duplicate 'name' columns if a simple merge was used.
# We prefer the 'name' from the EM calculation, or the DM if the EM name is null/NA
df_combined_indices['name'] = df_combined_indices['name_em'].fillna(df_combined_indices['name_dm'])

# Drop the redundant name columns
df_combined_indices = df_combined_indices.drop(columns=['name_em', 'name_dm'], errors='ignore')

# Filter out contributors with zero in both indices for a cleaner plot, unless they are all zeros
# Check if there's any non-zero value at all to decide on filtering
if (df_combined_indices['EM'].sum() > 0) or (df_combined_indices['DM'].sum() > 0):
    df_combined_indices_plot = df_combined_indices[
        (df_combined_indices['EM'] > 0) |
        (df_combined_indices['DM'] > 0)
    ].copy()
else:
    df_combined_indices_plot = df_combined_indices.copy() # Keep all if all are zero

# --- Add the aggregated project EM and DM point ---
# Ensure project_em and project_dm are available
if 'project_em' not in globals():
    # Fallback if not defined (should come from em-agregado cell)
    project_em = 0.0
if 'project_dm' not in globals():
    # Fallback if not defined (should come from dm-agregado cell)
    project_dm = 0.0

# Create a DataFrame for the aggregated point
df_aggregated_point = pd.DataFrame({
    'github_login': ['Project_Aggregated'],
    'name': ['Project'], # Changed text here
    'EM': [project_em],
    'DM': [project_dm]
})

# Concatenate with the existing DataFrame
df_combined_indices_plot = pd.concat([df_combined_indices_plot, df_aggregated_point], ignore_index=True)

# Create the scatter plot
fig = px.scatter(
    df_combined_indices_plot,
    x='DM', # DM on X-axis
    y='EM', # EM on Y-axis
    text=None, # Removed github_login as text on points
    hover_name='name', # Show full name on hover
    hover_data={
        'github_login': True, # Keep github_login in hover
        'name': False,       # Don't duplicate hover_name
        'EM': ':.2f',       # Format EM in hover
        'DM': ':.2f' # Format DM in hover
    },
    title='Contribution chart (EM vs DM)',
    labels={
        'EM': 'Engagement Magnitude (EM)',
        'DM': 'Delivery Magnitude (DM)'
    },
    # Removed 'size' from px.scatter to control it via update_traces
    color='github_login', # Assign a unique color to each contributor and the aggregated point
    symbol='github_login' # Assign a unique symbol to each contributor and the aggregated point
)

# Set a fixed, slightly larger size for all individual markers
fig.update_traces(marker=dict(size=10))

# Apply specific styling for the aggregated point
fig.update_traces(
    marker=dict(symbol='hexagon-dot', size=20, color='black', line=dict(width=2, color='white')),
    selector=dict(name='Project_Aggregated')
)

# Add text label for the aggregated point
fig.add_annotation(
    x=project_dm,
    y=project_em,
    text="Project", # Changed text here
    showarrow=False,
    xshift=0,
    yshift=15,
    font=dict(color='black', size=12, weight='bold')
)

# Add lines for quadrants
fig.add_hline(y=0.5, line_dash='dash', line_color='gray', annotation_text="Average EM")
fig.add_vline(x=0.5, line_dash='dash', line_color='gray', annotation_text="Average DM")

# Update layout for better aesthetics and legend position
fig.update_layout(
    height=900, # Increased height to accommodate the legend below
    width=900,
    xaxis=dict(range=[-0.05, 1.05]), # Extend range slightly beyond 0-1
    yaxis=dict(range=[-0.05, 1.05]),
    showlegend=True, # Ensure legend is shown
    legend=dict(
        orientation='v', # Vertical orientation
        yanchor='top',   # Anchor to the top of the legend container
        y=1,             # Position at the top right (y=1) of the plot area
        xanchor='right',
        x=1             # Position slightly outside the plot area to the right
    )
)

fig.show()
save_fig(fig, 'engagement_delivery_map.html')

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/engagement_delivery_map.html


* Add REMI e RDMI to dataframe

In [55]:
import pandas as pd

# Select only the necessary columns from df_combined_indices
df_indices_to_merge = df_combined_indices[['github_login', 'EM', 'DM']]

# Ensure there are no existing BCI or Code_Delivery_Index columns in df_people
# that would cause suffixing issues during merge. This is a more robust way to clean up.
cols_to_remove = [col for col in df_people.columns if col.startswith('BCI') or col.startswith('Code_Delivery_Index') or col.startswith('EM') or col.startswith('DM')]
if cols_to_remove:
    df_people = df_people.drop(columns=cols_to_remove)

# Merge with df_people to add EM and DM cleanly
df_people = pd.merge(df_people, df_indices_to_merge,
                     on='github_login',
                     how='left') # Use left merge to keep all rows from df_people

# Fill NaN values for EM and DM with 0 for contributors who had no activity
# These lines should now succeed because the merge would have added the columns.
df_people['EM'] = df_people['EM'].fillna(0.0)
df_people['DM'] = df_people['DM'].fillna(0.0)

print("df_people DataFrame com EM e DM adicionados:")
display(df_people.head())


df_people DataFrame com EM e DM adicionados:


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count,pull_requests_opened_count,pull_requests_merged_count,pull_requests_closed_count,total_additions,total_deletions,scaled_additions,scaled_deletions,EM,DM
0,monacofj,Monaco F. J.,yes,1,0,0,0,0,0,0,0,0.000000,0.000000,0.022097,0.000000
1,LiviaLelis,Lívia Lelis,yes,2,0,8,1,1,1,1832,250,6.106667,0.833333,0.262351,0.187044
2,trqt,Guilherme Torquato,yes,10,7,18,4,4,6,486,29,1.620000,0.096667,0.838339,0.405933
3,jotaefepinho,João Francisco Pinho,yes,1,0,2,0,0,2,0,0,0.000000,0.000000,0.066291,0.000000
4,chonkachu,N/A,yes,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000


## Engagement-Delivery distribution


### Engagement (EM) and Delivery (DM) stacked

In [56]:
## Plot Engagement (EM) and Delivery (DM) as a stacked bar chart.

import plotly.express as px
import pandas as pd

# Select relevant columns from df_people
df_plot_indices = df_people[['github_login', 'name', 'EM', 'DM']].copy()

# Filter out developers who have zero for both EM and DM
df_plot_indices_filtered = df_plot_indices[
    (df_plot_indices['EM'] > 0) | (df_plot_indices['DM'] > 0)
].copy()

if df_plot_indices_filtered.empty:
    print("No contributors with non-zero EM or DM scores found to plot.")
else:
    # Calculate total score for sorting
    df_plot_indices_filtered['total_score'] = df_plot_indices_filtered['EM'] + df_plot_indices_filtered['DM']

    # Sort df_plot_indices_filtered by total_score in descending order
    df_plot_indices_filtered = df_plot_indices_filtered.sort_values(by='total_score', ascending=False)

    # Melt the DataFrame to a long format suitable for stacked bar charts
    df_melted_indices = df_plot_indices_filtered.melt(
        id_vars=['github_login', 'name', 'total_score'], # Include total_score for potential hover or sorting
        value_vars=['EM', 'DM'],
        var_name='Metric',
        value_name='Value'
    )

    # Rename metrics for better readability in the plot
    df_melted_indices['Metric'] = df_melted_indices['Metric'].replace({
        'EM': 'EM (Engagement)',
        'DM': 'DM (Delivery)'
    })

    # Custom order for metrics so EM is 'em baixo' and DM is 'acima'
    metric_order = ['EM (Engagement)', 'DM (Delivery)']
    df_melted_indices['Metric'] = pd.Categorical(df_melted_indices['Metric'], categories=metric_order, ordered=True)

    # Get the sorted list of github_login for xaxis_categoryarray based on total_score
    sorted_github_logins = df_plot_indices_filtered['github_login'].tolist()

    # Create the stacked bar chart using Plotly Express
    fig = px.bar(
        df_melted_indices,
        x='github_login',
        y='Value',
        color='Metric',
        barmode='stack', # This creates stacked bars for each github_login
        title='Contribution rank',
        labels={
            'github_login': 'Contributor',
            'Value': 'Score',
            'Metric': 'Index Type'
        },
        hover_name='name',
        hover_data={
            'github_login': True, # Keep github_login in hover for context
            'name': False,       # Don't duplicate hover_name
            'Metric': True,
            'Value': ':.2f'      # Format Value to 2 decimal places in hover
        },
        text_auto='.2f' # Display text values automatically
    )

    # Ensure project_em and project_dm are available
    if 'project_em' not in globals():
        project_em = 0.0 # Fallback
    if 'project_dm' not in globals():
        project_dm = 0.0 # Fallback

    # Add horizontal line for aggregated EM (blue)
    fig.add_hline(y=project_em, line_width=1, line_dash="dash", line_color="blue",
                  annotation_text=f"Project EM: {project_em:.2f}",
                  annotation_position="top left",
                  annotation_font_color="blue")

    # Add horizontal line for aggregated DM (red) at its own value
    fig.add_hline(y=project_dm, line_width=1, line_dash="dash", line_color="red",
                  annotation_text=f"Project DM: {project_dm:.2f}",
                  annotation_position="bottom right", # Position below the line
                  annotation_font_color="red")

    # Add horizontal line for aggregated EM + DM (total project score) (black)
    fig.add_hline(y=project_em + project_dm, line_width=2, line_dash="dash", line_color="black",
                  annotation_text=f"Project Total (EM+DM): {project_em + project_dm:.2f}",
                  annotation_position="top right", # Position above the line
                  annotation_font_color="black")


    # Refine plot aesthetics
    fig.update_layout(
        xaxis_title='Contributor',
        yaxis_title='Score',
        legend_title='Index Type',
        xaxis_tickangle=90, # Rotate x-axis labels for readability
        bargap=0.1, # Gap between groups of bars (not very relevant for stacked, but good practice)
        height=600, # Adjust plot height
        xaxis_categoryorder='array', # Preserve the order from the DataFrame
        xaxis_categoryarray=sorted_github_logins
    )

    fig.update_yaxes(rangemode='tozero') # Ensure y-axis starts at zero

    # Set text color to white for all traces and position text inside
    fig.update_traces(textfont_color='white', textposition='inside')

    fig.show()

    save_fig(fig, 'contribution_em_dm_plot.html')

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/contribution_em_dm_plot.html


## Contribution radar

### Contribution score

In [57]:
## Calculate 'activity', 'reward', and 'score' for each developer.

import pandas as pd

# 1. Calculate 'activity': A_factor * (EM>0) + (1-A_factor) * (DM>0)
# Convert boolean to integer (True=1, False=0) for multiplication
df_people['activity'] = (df_people['EM'] > 0).astype(int) * A_factor + \
                        (df_people['DM'] > 0).astype(int) * (1 - A_factor)

# 2. Calculate 'reward' using the formula: R_factor * EM + (1-R_factor) * DM
df_people['reward'] = (R_factor * df_people['EM']) + ((1 - R_factor) * df_people['DM'])

# 3. Calculate the 'score' column as S_factor * activity + (1 - S_factor) * reward (range 0-1)
df_people['score'] = S_factor * df_people['activity'] + (1 - S_factor) * df_people['reward']

# Display the head of the df_people DataFrame to show the newly added 'activity', 'reward', and 'score' columns.
display(df_people.head())

,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count,pull_requests_opened_count,pull_requests_merged_count,pull_requests_closed_count,total_additions,total_deletions,scaled_additions,scaled_deletions,EM,DM,activity,reward,score
0,monacofj,Monaco F. J.,yes,1,0,0,0,0,0,0,0,0.000000,0.000000,0.022097,0.000000,0.5,0.011049,0.255524
1,LiviaLelis,Lívia Lelis,yes,2,0,8,1,1,1,1832,250,6.106667,0.833333,0.262351,0.187044,1.0,0.224697,0.612349
2,trqt,Guilherme Torquato,yes,10,7,18,4,4,6,486,29,1.620000,0.096667,0.838339,0.405933,1.0,0.622136,0.811068
3,jotaefepinho,João Francisco Pinho,yes,1,0,2,0,0,2,0,0,0.000000,0.000000,0.066291,0.000000,0.5,0.033146,0.266573
4,chonkachu,N/A,yes,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


### Contribution score (CS) plot

In [75]:
## Contribution score (CS) plot

import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots # Import make_subplots

# Ensure 'activity', 'reward', and 'score' columns are already calculated in df_people

# --- Calculate Project-level Activity, Reward, and Score ---
# project_em and project_dm are already calculated globally (0-1 scaled)
# A_factor and R_factor are from the score-parameters cell

# Ensure project_em and project_dm are numerical (they are floats from numpy, but good practice to check)
project_em_val = project_em if pd.notna(project_em) else 0.0
project_dm_val = project_dm if pd.notna(project_dm) else 0.0

project_activity = A_factor * (1 if project_em_val > 0 else 0) + (1 - A_factor) * (1 if project_dm_val > 0 else 0)
project_reward = R_factor * project_em_val + (1 - R_factor) * project_dm_val
project_score = S_factor * project_activity + (1 - S_factor) * project_reward

print(f"Calculated Project Activity: {project_activity:.2f}")
print(f"Calculated Project Reward: {project_reward:.2f}")
print(f"Calculated Project Score: {project_score:.2f}")

# Calculate the four individual components of the score
df_people['score_component_activity_em_present'] = S_factor * A_factor * (df_people['EM'] > 0).astype(int)
df_people['score_component_activity_dm_present'] = S_factor * (1 - A_factor) * (df_people['DM'] > 0).astype(int)
df_people['score_component_reward_em_magnitude'] = (1 - S_factor) * R_factor * df_people['EM']
df_people['score_component_reward_dm_magnitude'] = (1 - S_factor) * (1 - R_factor) * df_people['DM']

# 1. Create a new DataFrame, df_contribution_plot, using the score components
df_contribution_plot = df_people[[
    'github_login', 'name', 'score',
    'score_component_activity_em_present',
    'score_component_activity_dm_present',
    'score_component_reward_em_magnitude',
    'score_component_reward_dm_magnitude'
]].copy()

# 2. Filter df_contribution_plot to include only contributors where score > 0
df_contribution_plot_individuals = df_contribution_plot[df_contribution_plot['score'] > 0].copy()

# 3. Melt df_contribution_plot into a long format suitable for stacking
df_melted_score_individuals = df_contribution_plot_individuals.melt(
    id_vars=['github_login', 'name', 'score'],
    value_vars=[
        'score_component_activity_em_present',
        'score_component_activity_dm_present',
        'score_component_reward_em_magnitude',
        'score_component_reward_dm_magnitude'
    ],
    var_name='Component Type',
    value_name='Value'
)

# --- Add Project-level score as a bar ---
project_score_component_activity_em_present = S_factor * A_factor * (1 if project_em_val > 0 else 0)
project_score_component_activity_dm_present = S_factor * (1 - A_factor) * (1 if project_dm_val > 0 else 0)
project_score_component_reward_em_magnitude = (1 - S_factor) * R_factor * project_em_val
project_score_component_reward_dm_magnitude = (1 - S_factor) * (1 - R_factor) * project_dm_val

# Define raw component order for consistent stacking
raw_component_order = [
    'score_component_activity_em_present',
    'score_component_activity_dm_present',
    'score_component_reward_em_magnitude',
    'score_component_reward_dm_magnitude'
]

project_data_for_bar = {
    'github_login': ['Project'] * len(raw_component_order),
    'name': ['Project'] * len(raw_component_order),
    'score': [project_score] * len(raw_component_order),
    'Component Type': raw_component_order,
    'Value': [
        project_score_component_activity_em_present,
        project_score_component_activity_dm_present,
        project_score_component_reward_em_magnitude,
        project_score_component_reward_dm_magnitude
    ]
}
df_project_score_bar = pd.DataFrame(project_data_for_bar)

# Add a blank space as a separator for better visual spacing
separator_label = '  ' # Using two spaces for a slightly wider string, which translates to a wider category
df_separator_melted = pd.DataFrame({
    'github_login': [separator_label] * len(raw_component_order),
    'name': [separator_label] * len(raw_component_order),
    'score': [0.0] * len(raw_component_order),
    'Component Type': raw_component_order,
    'Value': [0.0] * len(raw_component_order)
})

# Concatenate all melted dataframes (individuals, separator, project) with raw component names
df_melted_score_all = pd.concat([
    df_melted_score_individuals,
    df_separator_melted,
    df_project_score_bar
], ignore_index=True)

# --- Determine the correct sorting order for all bars (individuals + Project) ---
# 1. Get total scores for individuals for sorting
df_individual_scores_for_sort = df_melted_score_individuals[['github_login', 'score']].drop_duplicates()

# 2. Sort individuals by score in descending order
sorted_individual_logins = df_individual_scores_for_sort.sort_values(by='score', ascending=False)['github_login'].tolist()

# 3. Manually construct the final sorted list including separator and 'Project' at the end
sorted_github_logins_for_score = sorted_individual_logins + [separator_label, 'Project']


# Re-order the concatenated DataFrame using the determined order of logins
df_melted_score = df_melted_score_all.set_index('github_login').loc[sorted_github_logins_for_score].reset_index()

# Similar re-ordering for df_contribution_plot (for text labels)
df_project_contribution_plot_for_labels = df_project_contribution_plot # Rename to avoid confusion with df_project_score_bar
df_separator_contribution = pd.DataFrame([{
    'github_login': separator_label,
    'name': separator_label,
    'score': 0.0,
    'score_component_activity_em_present': 0.0,
    'score_component_activity_dm_present': 0.0,
    'score_component_reward_em_magnitude': 0.0,
    'score_component_reward_dm_magnitude': 0.0
}])

df_contribution_plot_all = pd.concat([
    df_contribution_plot_individuals,
    df_separator_contribution,
    df_project_contribution_plot_for_labels
], ignore_index=True)

df_contribution_plot = df_contribution_plot_all.set_index('github_login').loc[sorted_github_logins_for_score].reset_index()

# Now, rename component types for better plot labels in the final df_melted_score
df_melted_score['Component Type'] = df_melted_score['Component Type'].replace({
    'score_component_activity_em_present': 'Activity (EM Present)',
    'score_component_activity_dm_present': 'Activity (DM Present)',
    'score_component_reward_em_magnitude': 'Reward (EM Magnitude)',
    'score_component_reward_dm_magnitude': 'Reward (DM Magnitude)'
})

# Define custom colors for the components
# Two shades of blue for activity components, two shades of green for reward components
component_colors = {
    'Activity (EM Present)': '#2A52BE',  # Darker Blue
    'Activity (DM Present)': '#6495ED',  # Slightly darker CornflowerBlue
    'Reward (EM Magnitude)': '#228B22',  # Darker Green (ForestGreen)
    'Reward (DM Magnitude)': '#66CDAA'   # Slightly darker MediumAquamarine
}

# Apply categorical order after replacement for correct legend and stacking order.
df_melted_score['Component Type'] = pd.Categorical(df_melted_score['Component Type'], categories=[
    'Activity (EM Present)',
    'Activity (DM Present)',
    'Reward (EM Magnitude)',
    'Reward (DM Magnitude)'
], ordered=True)


# 6. Create a stacked bar chart using Plotly Express
fig = go.Figure()

# Add bar traces for individual score components
for component in [
    'Activity (EM Present)',
    'Activity (DM Present)',
    'Reward (EM Magnitude)',
    'Reward (DM Magnitude)'
]:
    df_component = df_melted_score[df_melted_score['Component Type'] == component]

    # Determine color based on whether it's a project bar or individual bar
    current_colors = []
    for login in df_component['github_login']:
        if login == separator_label: # Make separator invisible
            current_colors.append('rgba(0,0,0,0)')
        else:
            # Always use the standard component colors for all bars, including 'Project'
            current_colors.append(component_colors[component])

    fig.add_trace(go.Bar(
        x=df_component['github_login'],
        y=df_component['Value'],
        name=component,
        marker_color=current_colors,
        text=[f'{v:.2f}' if v > 0 else '' for v in df_component['Value']], # Add text values, hide for 0
        hovertemplate=
            "<b>%{customdata[1]}</b> (%{x})<br>" +
            "Component Type: %{name}<br>" +
            "Value: %{y:.2f}<br>" +
            "Total Score: %{customdata[0]:.2f}<extra></extra>", # customdata[0] for total score
        customdata=df_component[['score', 'name']].values, # Pass score and name for hover as a numpy array
        showlegend=True,
    ))


# Add total score as text on top of each bar (primary y-axis)
fig.add_trace(go.Scatter(
    x=df_contribution_plot['github_login'],
    y=df_contribution_plot['score'],
    mode='text',
    text=[f'{s:.1f}' if s > 0 else '' for s in df_contribution_plot['score']], # Format total score to 1 decimal place, hide for 0
    textposition='top center',
    textfont=dict(color='black', size=10),
    showlegend=False,
    hoverinfo='none',
))


# f. Adjust the x-axis to show labels vertically and preserve the order from the DataFrame
custom_tick_text = [login if login != separator_label else '' for login in sorted_github_logins_for_score]

fig.update_xaxes(tickangle=-90, tickfont=dict(size=10),
                 categoryorder='array',
                 categoryarray=sorted_github_logins_for_score,
                 ticktext=custom_tick_text, # Use the custom tick text
                 tickvals=sorted_github_logins_for_score) # Keep tickvals for positioning

# g. Update y-axes properties
fig.update_yaxes(
    rangemode='tozero',
    title_text='Score Component Value',
    range=[0, 1.1] # Adjusted range to 0-1.1
)


# Update layout for title and overall settings
fig.update_layout(
    title_text='Contribution score breakdown (with Project Total)', # Updated title
    barmode='stack', # This is important for go.Bar traces to stack
    height=700,
    bargap=0.3, # Increased gap between bars to create more separation
    legend=dict(
        orientation='v', # Vertical orientation
        yanchor='middle', # Anchor to the middle
        y=0.5,             # Position vertically centered (0.5 means middle of plot area)
        xanchor='left',
        x=1.02             # Position slightly outside the plot area to the right
    )
)

# Display the Value on top of the bars in white text, positioned inside (for components)
fig.update_traces(textfont_color='white', textposition='inside', selector=dict(type='bar'))

fig.show()

print("Overall Score calculation and stacked bar chart generation complete.")
save_fig(fig, 'score_plot.html')

Calculated Project Activity: 1.00
Calculated Project Reward: 0.22
Calculated Project Score: 0.61


Overall Score calculation and stacked bar chart generation complete.
Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/score_plot.html


In [60]:
# cellname : score-evolution
import pandas as pd
import plotly.express as px
from datetime import date, timedelta, datetime, timezone
import numpy as np

# Ensure start_date and selected_timedelta are available (from previous cells)
# start_date for event fetching is a datetime with timezone.
# We need to get the date part for comparison with dictionary keys.

# The start_date for fetching events is already timezone-aware
# start_date is defined in bd46d806 or earlier cells
if 'start_date' not in globals():
    start_date = datetime.now(timezone.utc) - timedelta(weeks=1) # Fallback to 1 week if not defined

end_date_for_range = datetime.now(timezone.utc).date() # Current date
start_date_for_range = start_date.date() # Date part of the fetching start_date

all_dates = [start_date_for_range + timedelta(days=x) for x in range((end_date_for_range - start_date_for_range).days + 1)]

daily_project_metrics = []

# Ensure max_discussion, max_prs_opened, max_code_changes, max_prs_merged are available
# from previous cells like 'em-agregado' and 'dm-agregado'
# Fallback values if not found (should be defined if previous cells were run)
if 'max_discussion' not in globals() or max_discussion == 0: max_discussion_val = 1
else: max_discussion_val = max_discussion

if 'max_prs_opened' not in globals() or max_prs_opened == 0: max_prs_opened_val = 1
else: max_prs_opened_val = max_prs_opened

if 'max_code_changes' not in globals() or max_code_changes == 0: max_code_changes_val = 1
else: max_code_changes_val = max_code_changes

if 'max_prs_merged' not in globals() or max_prs_merged == 0: max_prs_merged_val = 1
else: max_prs_merged_val = max_prs_merged

# Ensure A_factor, R_factor, S_factor are available
if 'A_factor' not in globals(): A_factor = 0.5
if 'R_factor' not in globals(): R_factor = 0.5
if 'S_factor' not in globals(): S_factor = 0.5

# Defensive check: Ensure daily_additions and daily_deletions are dictionaries
# This guards against cases where prior cells might not have been run or
# if they were inadvertently reassigned as integers.
if not isinstance(daily_additions, dict):
    daily_additions = {}
if not isinstance(daily_deletions, dict):
    daily_deletions = {}

# Fallback values for theoretical maximums, if not defined globally
if 'active_contributors_for_em_count' not in globals() or active_contributors_for_em_count == 0:
    active_contributors_for_em_count_val = 1
else:
    active_contributors_for_em_count_val = active_contributors_for_em_count

if 'max_theoretical_project_em_raw' not in globals() or max_theoretical_project_em_raw == 0:
    max_theoretical_project_em_raw_val = active_contributors_for_em_count_val * np.sqrt(2)
else:
    max_theoretical_project_em_raw_val = max_theoretical_project_em_raw

if 'active_contributors_for_dm_count' not in globals() or active_contributors_for_dm_count == 0:
    active_contributors_for_dm_count_val = 1
else:
    active_contributors_for_dm_count_val = active_contributors_for_dm_count

if 'max_theoretical_project_dm_raw' not in globals() or max_theoretical_project_dm_raw == 0:
    max_theoretical_project_dm_raw_val = active_contributors_for_dm_count_val * np.sqrt(2)
else:
    max_theoretical_project_dm_raw_val = max_theoretical_project_dm_raw


for current_date in all_dates:
    # Get daily event counts
    daily_issues_raised = daily_issues_raised_count.get(current_date, 0)
    daily_issue_comments = daily_issue_comment_count.get(current_date, 0)
    daily_prs_opened = daily_prs_opened_count.get(current_date, 0)
    daily_prs_merged = daily_prs_merged_count.get(current_date, 0)

    # Use distinct variable names to avoid overwriting the global dictionaries
    current_day_additions = daily_additions.get(current_date, 0)
    current_day_deletions = daily_deletions.get(current_date, 0)

    # Calculate daily total discussion and code changes
    daily_total_discussion = daily_issues_raised + daily_issue_comments
    daily_total_code_changes = current_day_additions + current_day_deletions

    # Normalize daily EM components by global individual maximums
    daily_normalized_discussion = daily_total_discussion / max_discussion_val
    daily_normalized_prs_opened = daily_prs_opened / max_prs_opened_val

    # Calculate daily EM (scaled 0-1) using the theoretical project maximum
    daily_em_raw = np.sqrt(daily_normalized_discussion**2 + daily_normalized_prs_opened**2)
    daily_em = 0.0
    if max_theoretical_project_em_raw_val > 0:
        daily_em = daily_em_raw / max_theoretical_project_em_raw_val
    # Ensure it's capped at 1.0 to handle extreme cases, though with correct scaling it should be <= 1
    daily_em = min(daily_em, 1.0)

    # Normalize daily DM components by global individual maximums
    daily_normalized_code_changes = daily_total_code_changes / max_code_changes_val
    daily_normalized_prs_merged = daily_prs_merged / max_prs_merged_val

    # Calculate daily DM (scaled 0-1) using the theoretical project maximum
    daily_dm = 0.0
    daily_dm_raw = np.sqrt(daily_normalized_code_changes**2 + daily_normalized_prs_merged**2)
    if max_theoretical_project_dm_raw_val > 0:
        daily_dm = daily_dm_raw / max_theoretical_project_dm_raw_val
    # Ensure it's capped at 1.0
    daily_dm = min(daily_dm, 1.0)

    # Calculate daily Activity
    daily_activity = A_factor * (1 if daily_em > 0 else 0) + (1 - A_factor) * (1 if daily_dm > 0 else 0)

    # Calculate daily Reward
    daily_reward = R_factor * daily_em + (1 - R_factor) * daily_dm

    # Calculate daily Contribution Score (CS)
    daily_cs = S_factor * daily_activity + (1 - S_factor) * daily_reward

    # Calculate the four individual components of the daily score
    daily_score_component_activity_em_present = S_factor * A_factor * (1 if daily_em > 0 else 0)
    daily_score_component_activity_dm_present = S_factor * (1 - A_factor) * (1 if daily_dm > 0 else 0)
    daily_score_component_reward_em_magnitude = (1 - S_factor) * R_factor * daily_em
    daily_score_component_reward_dm_magnitude = (1 - S_factor) * (1 - R_factor) * daily_dm

    daily_project_metrics.append({
        'Date': current_date,
        'Activity (EM Present)': daily_score_component_activity_em_present,
        'Activity (DM Present)': daily_score_component_activity_dm_present,
        'Reward (EM Magnitude)': daily_score_component_reward_em_magnitude,
        'Reward (DM Magnitude)': daily_score_component_reward_dm_magnitude,
        'Score': daily_cs # Changed 'CS' to 'Score'
    })

df_daily_project_metrics = pd.DataFrame(daily_project_metrics)
df_daily_project_metrics['Date'] = pd.to_datetime(df_daily_project_metrics['Date'])

# Melt the DataFrame for line chart visualization
df_daily_melted_metrics = df_daily_project_metrics.melt(
    id_vars=['Date'],
    value_vars=['Activity (EM Present)', 'Activity (DM Present)', 'Reward (EM Magnitude)', 'Reward (DM Magnitude)', 'Score'], # Changed 'CS' to 'Score'
    var_name='Metric',
    value_name='Value'
)

# Create the smoothed line chart using Plotly Express
fig = px.line(
    df_daily_melted_metrics,
    x='Date',
    y='Value',
    color='Metric',
    title='Daily Evolution of Project Metrics',
    labels={'Value': 'Metric Value', 'Date': 'Date'},
    hover_data={'Value': ':.2f'},
    line_shape='spline'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Metric Value',
    legend_title='Metric',
    hovermode='x unified'
    # Removed yaxis_range=[0, 1.1] to allow automatic adjustment
)

fig.update_xaxes(dtick="D1", tickformat="%b %d") # Daily ticks, format as "Month Day"

fig.show()

save_fig(fig, 'daily_project_metrics_evolution.html')

print("Daily project metrics evolution chart generated.")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/daily_project_metrics_evolution.html
Daily project metrics evolution chart generated.


### Score percentiles

In [61]:
## Plot contribution score percentiles with a box plot.

import plotly.express as px
import pandas as pd

# Filter out contributors with a score of 0 for a more meaningful quartile analysis
df_active_scores = df_people[df_people['score'] > 0].copy()

if df_active_scores.empty:
    print("No contributors with a score > 0 found to plot quartiles.")
else:
    # Create a box plot for the 'score'
    fig = px.box(
        df_active_scores,
        y='score',
        title='Contribution score percentiles',
        labels={
            'score': 'Contribution Score'
        },
        hover_data={'github_login': True, 'name': True, 'score': ':.2f'}
    )

    # Customize layout for better readability
    fig.update_layout(
        yaxis_title='Score',
        yaxis_range=[0, 1], # Ensure y-axis covers the full score range from 0 to 1
        yaxis=dict(dtick=0.1), # Set grid lines every 0.1 units
        boxmode='overlay' # Overlay points if multiple boxes (not applicable for single score column)
    )

    # Add individual data points as a scatter plot overlay (optional, for detail)
    fig.add_trace(px.scatter(
        df_active_scores,
        y='score',
        hover_name='name',
        hover_data={'github_login': True, 'name': True, 'score': ':.2f'}
    ).data[0])

    fig.show()
    save_fig(fig, 'score_percentiles_box_plot.html')

    print("Box plot for contributor scores generated.")

    # Print quartile values with explanations
    q1 = df_active_scores['score'].quantile(0.25)
    median = df_active_scores['score'].quantile(0.50)
    q3 = df_active_scores['score'].quantile(0.75)
    max_score = df_active_scores['score'].max()
    min_score = df_active_scores['score'].min()

    print("\n--- Contributor Score Quartiles ---")
    print(f"* The lowest active score is: {min_score:.2f}")
    print(f"* 25% of contributors have a score up to: {q1:.2f}")
    print(f"* 50% of contributors (the median) have a score up to: {median:.2f}")
    print(f"* 75% of contributors have a score up to: {q3:.2f}")
    print(f"* The highest active score is: {max_score:.2f}")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/score_percentiles_box_plot.html
Box plot for contributor scores generated.

--- Contributor Score Quartiles ---
* The lowest active score is: 0.26
* 25% of contributors have a score up to: 0.29
* 50% of contributors (the median) have a score up to: 0.54
* 75% of contributors have a score up to: 0.61
* The highest active score is: 0.87


### Score histogram

In [62]:
## Plot contribution score distribution with a histogram.

import plotly.express as px
import pandas as pd

# Filter out contributors with a score of 0 for a more meaningful histogram
df_active_scores_hist = df_people[df_people['score'] > 0].copy()

if df_active_scores_hist.empty:
    print("No contributors with a score > 0 found to plot a histogram.")
else:
    # Create a histogram for the 'score'
    fig = px.histogram(
        df_active_scores_hist,
        x='score',
        nbins=20, # Adjust number of bins as needed
        title='Contribution score histogram',
        labels={
            'score': 'Contribution Score',
            'count': 'Number of Contributors'
        },
        hover_data={'name': True, 'score': ':.2f'}
    )

    # Customize layout for better readability
    fig.update_layout(
        xaxis_title='Contribution Score',
        yaxis_title='Number of Contributors',
        xaxis_range=[0, 1], # Ensure x-axis covers the full score range
        bargap=0.1 # Gap between bars
    )

    # Calculate mean and median
    mean_score = df_active_scores_hist['score'].mean()
    median_score = df_active_scores_hist['score'].median()

    # Add mean line
    fig.add_vline(x=mean_score, line_width=2, line_dash="dash", line_color="red",
                  annotation_text=f"Mean: {mean_score:.2f}",
                  annotation_position="top right")

    # Add median line
    fig.add_vline(x=median_score, line_width=2, line_dash="dot", line_color="green",
                  annotation_text=f"Median: {median_score:.2f}",
                  annotation_position="top left")

    fig.show()

    print("Histogram for contributor scores generated.")
    save_fig(fig, 'score_histogram.html')

Histogram for contributor scores generated.
Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/score_histogram.html


### Score proportions

In [63]:
# cellname : scopre-pizza
import plotly.express as px
import pandas as pd

# 1. Categorize the 'score' column with new names
def categorize_score(score):
    if score == 0:
        return 'Zero (score = 0)'
    elif 0 < score < 0.5:
        return 'Low (0 < score < 0.5)'
    else:
        return 'Good (0.5 <= score)'

df_people['score_category'] = df_people['score'].apply(categorize_score)

# 2. Count the occurrences in each category
score_counts = df_people['score_category'].value_counts().reset_index()
score_counts.columns = ['Category', 'Count']

# 3. Define custom colors for the categories
category_colors = {
    'Zero (score = 0)': '#aaaa80',       # Lighter gray
    'Low (0 < score < 0.5)': 'darkred',    # Darker red
    'Good (0.5 <= score)': 'darkcyan'            # Cyan
}

# Ensure category order for consistent plotting
category_order = ['Good (0.5 <= score)', 'Low (0 < score < 0.5)', 'Zero (score = 0)']
score_counts['Category'] = pd.Categorical(score_counts['Category'], categories=category_order, ordered=True)
score_counts = score_counts.sort_values('Category')

# 5. Create the pie chart
fig = px.pie(
    score_counts,
    values='Count',
    names='Category', # Use the raw category name for slice identification
    title='Contribution score ranges',
    color='Category',
    color_discrete_map=category_colors,
    hover_data=['Count', 'Category']
)

# Update traces to show percentage and formatted name as text, outside
fig.update_traces(textposition='outside', textinfo='percent', texttemplate="<b>%{label}</b><br>%{percent:.1%}<br>(%{value})", textfont_color='black')

fig.show()
save_fig(fig, 'score_proportions_pie_chart.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/score_proportions_pie_chart.html


### Indices distribution

In [64]:
## Plot quartile distribution of tide metrics with box plots.

import pandas as pd
import plotly.express as px
import numpy as np

# Prepare the data for quartile analysis
# Use the original activity and reward values, which are already normalized 0-1
df_people_quartiles = df_people.copy()

# Select the relevant columns for the box plots
metrics_for_quartiles = df_people_quartiles[['github_login', 'EM', 'DM', 'activity', 'reward']]

# Filter out contributors who have 0 across all these metrics for meaningful quartiles
df_active_metrics = metrics_for_quartiles[
    (metrics_for_quartiles['EM'] > 0) | \
    (metrics_for_quartiles['DM'] > 0) | \
    (metrics_for_quartiles['activity'] > 0) | \
    (metrics_for_quartiles['reward'] > 0)
].copy()

if df_active_metrics.empty:
    print("No active contributors found to display quartile distributions.")
else:
    # Melt the DataFrame to a long format suitable for px.box for side-by-side plots
    df_melted_metrics = df_active_metrics.melt(
        id_vars=['github_login'],
        value_vars=['EM', 'DM', 'activity', 'reward'],
        var_name='Metric',
        value_name='Value'
    )

    # Rename metrics for better display in the plot
    df_melted_metrics['Metric'] = df_melted_metrics['Metric'].replace({
        'EM': 'EM (Engagement)',
        'DM': 'DM (Delivery)',
        'activity': 'Activity',
        'reward': 'Reward'
    })

    # Define explicit order for metrics for consistent plotting
    metric_order = ['EM (Engagement)', 'DM (Delivery)', 'Activity', 'Reward']
    df_melted_metrics['Metric'] = pd.Categorical(df_melted_metrics['Metric'], categories=metric_order, ordered=True)

    # Create the box plot
    fig = px.box(df_melted_metrics,
                 x='Metric',
                 y='Value',
                 color='Metric', # Color boxes by metric type
                 title='Quartile distribution of tide metrics',
                 labels={'Metric': 'Metric', 'Value': 'Value'}, # Updated labels to English
                 hover_data={'github_login': True, 'Value': ':.2f'},
                 points='all' # Display all individual points
                )

    # Customize layout
    fig.update_layout(
        xaxis_title='Metric',
        yaxis_title='Value',
        xaxis_tickangle=-45,
        height=600,
        showlegend=False, # Legend not needed as colors are explained by x-axis
        yaxis_range=[0, 1], # Set y-axis range to 0-1
        yaxis=dict(dtick=0.1)
    )

    fig.show()
    save_fig(fig, 'metric_quartiles_box_plot.html')

    print("\n--- Quartile Values for Each Metric ---")
    for metric in ['EM', 'DM', 'activity', 'reward']:
        series = df_active_metrics[metric]
        if not series.empty:
            q1 = series.quantile(0.25)
            median = series.quantile(0.50)
            q3 = series.quantile(0.75)
            max_val = series.max()
            min_val = series.min()
            mean_val = series.mean()

            print(f"\nMetric: {metric.replace('EM', 'Engagement Magnitude').replace('DM', 'Delivery Magnitude')}")
            print(f"  Mean: {mean_val:.2f}")
            print(f"  Min: {min_val:.2f}")
            print(f"  Q1 (25th percentile): {q1:.2f}")
            print(f"  Median (50th percentile): {median:.2f}")
            print(f"  Q3 (75th percentile): {q3:.2f}")
            print(f"  Max: {max_val:.2f}")
        else:
            print(f"\nMetric: {metric.replace('EM', 'Engagement Magnitude').replace('DM', 'Delivery Magnitude')} - No active data.")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-30/metric_quartiles_box_plot.html

--- Quartile Values for Each Metric ---

Metric: Engagement Magnitude
  Mean: 0.27
  Min: 0.02
  Q1 (25th percentile): 0.15
  Median (50th percentile): 0.19
  Q3 (75th percentile): 0.33
  Max: 0.84

Metric: Delivery Magnitude
  Mean: 0.11
  Min: 0.00
  Q1 (25th percentile): 0.00
  Median (50th percentile): 0.00
  Q3 (75th percentile): 0.10
  Max: 0.81

Metric: activity
  Mean: 0.78
  Min: 0.50
  Q1 (25th percentile): 0.50
  Median (50th percentile): 1.00
  Q3 (75th percentile): 1.00
  Max: 1.00

Metric: reward
  Mean: 0.19
  Min: 0.01
  Q1 (25th percentile): 0.07
  Median (50th percentile): 0.13
  Q3 (75th percentile): 0.21
  Max: 0.75


# OTHER INFORMATION

## Inactive developers

* Devs that didn't contribute dureing the period

In [65]:
## Identify and list inactive developers.

import pandas as pd
from IPython.display import display, Markdown

# Define the columns that represent contribution events
contribution_columns = [
    'issues_opened_count',
    'issues_reopened_count',
    'issue_comment_event_count',
    'pull_requests_opened_count',
    'pull_requests_merged_count',
    'pull_requests_closed_count',
    'total_additions',
    'total_deletions'
]

# Ensure all contribution columns exist in df_people
existing_contribution_columns = [col for col in contribution_columns if col in df_people.columns]

# Calculate the total contributions for each person
# Summing across relevant columns for each row
df_people['total_contributions'] = df_people[existing_contribution_columns].sum(axis=1)

# Filter for developers with zero total contributions
inactive_devs_df = df_people[df_people['total_contributions'] == 0]

# Extract logins of inactive developers for potential future use
inactive_dev_logins = inactive_devs_df['github_login'].tolist()

if not inactive_devs_df.empty:
    display(Markdown(f"**{len(inactive_devs_df)} Developer(s) with No Contributions in the Period:**"))
    for index, row in inactive_devs_df.iterrows():
        dev_name = row['name'] if row['name'] != 'N/A' else 'Name not available'
        display(Markdown(f"- {row['github_login']} ({dev_name})"))
else:
    display(Markdown("**All developers have made at least one contribution in the selected period!**"))

# Clean up the temporary column
df_people.drop(columns=['total_contributions'], inplace=True, errors='ignore')

**14 Developer(s) with No Contributions in the Period:**

- chonkachu (Name not available)

- liestrela (li)

- migueltomazini (Miguel Tomazini)

- AndreyCortez (Andrey Cortez)

...

- Peloro (Pedro Louro)

- GiordanoSL (Giordano Santorum Lorenzetto)

- lejonck (Name not available)

- ginoventa (Giovanna Noventa )

- gustavorspires (Gustavo Ramos Santos Pires)

# SAVE DATAFRAME

In [66]:
## Save the final df_people DataFrame to CSV.

import os

# Ensure output_folder and df_people are defined from previous cells

output_csv_path = os.path.join(output_folder, 'developers_data.csv')
df_people.to_csv(output_csv_path, index=False)

print(f"DataFrame 'df_people' successfully saved to {output_csv_path}")

DataFrame 'df_people' successfully saved to /content/gdrive/My Drive/naja-2025-11-30/developers_data.csv


# CREATE REPORT

## Section descriptions

In [67]:
# --- Report Description Variables ---

# Project Summary
summary_description = "This section provides a high-level overview of the GitHub repository, including its basic metadata and general statistics."

# Project Overview
overview_description = "This section summarizes the key demographic data for contributors and collaborators within the analyzed period."

# Inactive Developers
inactive_devs_description = "This section lists developers who are either collaborators or external contributors but did not record any activity (issues opened/reopened, comments, PRs opened/merged, code changes) within the selected analysis period."
inactive_devs_found_text = "Developer(s) with No Contributions in the Period:"
inactive_devs_none_text = "All developers have made at least one contribution in the selected period!"

# All Primitive Events
primitive_events_description = "This chart displays the raw count of various GitHub events (issues opened, issues reopened, comments, PRs opened, PRs merged, lines added, lines removed) per contributor. It provides a granular view of individual developer activity."

# All Aggregate Events
aggregate_events_description = "This chart aggregates primitive events into broader categories: 'Issues Raised' (opened + reopened), 'Comments' (on issues/PRs), 'PRs Opened', 'PRs Merged', and 'Code Changes' (additions + deletions). It offers a summarized view of contribution types."

# WB
wbi_description = "The WB (Workload Balance), calculated as (PRs Closed - PRs Opened) / (PRs Closed + PRs Opened), measures the balance between new work introduced (PRs opened) and work completed (PRs closed) in the project. A positive value indicates clearing the backlog, while a negative value suggests accumulation."

# RB
rbi_description = "The RB (Resolution Balance), calculated as (PRs Submitted - Issues Raised) / (PRs Submitted + Issues Raised), reflects the balance between problem identification (issues raised) and solution implementation (PRs submitted). A positive value implies a focus on implementation, while a negative value suggests more issues are being identified than solutions proposed."

# Project Performance Map
performance_map_description = "This map plots the Workload Balance (WB) against the Resolution Balance (RB) on a polar coordinate system. It visually represents the overall project dynamics, indicating whether the team is accumulating/clearing work and focusing on planning/implementing tasks."

# PRs Close Time section
pr_close_time_description = "This section analyzes the lifecycle of Pull Requests, specifically focusing on the time taken to close them. It categorizes PRs based on their creation and closure dates relative to the analysis period."

# Developers Overview - Engagement
engagement_plot_description = "This plot visualizes individual contributor engagement by mapping 'PRs Opened' against 'Discussion' (Issues Opened + Comments). It helps identify developers who are more conversational versus those focused on initiating code contributions."
remi_index_description = "EM quantifies a developer's overall engagement by taking the Euclidean norm of their normalized 'Discussion' and 'PRs Opened' scores. It intuitively measures the intensity of a developer's participatory and initiatory actions, ranging from 0 (no engagement) to 1 (maximum engagement)."

# Developers Overview - Delivery
delivery_plot_description = "This plot visualizes individual contributor delivery by mapping 'Code Changes' (lines added + deleted) against 'PRs Merged'. It helps identify developers who are generating significant code volume versus those focused on successfully integrating code into the main branch."
rdmi_index_description = "DM quantifies a developer's overall delivery by taking the Euclidean norm of their normalized 'Code Changes' and 'PRs Merged' scores. It intuitively measures the overall strength of a developer's effective code delivery, ranging from 0 (no delivery) to 1 (maximum delivery)."

# Engagement-Delivery Map
engagement_delivery_map_description = "This map plots each developer's EM against their DM score. It categorizes developers based on their relative engagement and delivery levels, helping identify different contributor profiles (e.g., highly engaged/low delivery vs. low engagement/high delivery)."

# Contribution (EM+DM) Plot
contribution_plot_description = "This stacked bar chart displays each developer's EM and DM scores side-by-side. It offers a combined view of individual engagement and delivery magnitudes, allowing for quick comparison across contributors."

# Score Plot
score_plot_description = "The overall score for each contributor is calculated as a weighted sum of 'Activity' and 'Reward', using the formula `S * Activity + (1 - S) * Reward`. This provides a comprehensive measure between 0 and 1. 'Activity' measures minimal participation, while 'Reward' measures participation intensity based on EM and DM. This chart displays the stacked components of 'Activity' and 'Reward' contributing to the total score."
score_percentiles_description = "This box plot visualizes the distribution of individual contributor scores, showing quartiles (25th, 50th, 75th percentiles), minimum, and maximum values. It helps in understanding the spread and central tendency of scores across active developers."
score_histogram_description = "This histogram shows the frequency distribution of contributor scores, indicating how many developers fall within specific score ranges. It helps identify patterns in scoring, such as concentration around certain values or presence of multiple peaks."
score_proportions_description = "This pie chart categorizes contributors into three score ranges: 'Zero' (score = 0), 'Low' (0 < score < 0.5), and 'Good' (0.5 <= score). It illustrates the proportion of developers in each category, providing a high-level overview of the team's overall performance distribution."

# Aggregate Indices
metric_quartiles_description = "This box plot displays the quartile distribution (25th, 50th, 75th percentiles) of individual EM, DM, Activity, and Reward scores across all active contributors. It provides insights into the spread and central tendency of these key metrics among developers."


## Report structure

In [68]:
## Generate the final HTML report.

import os
from datetime import datetime

# Ensure output_folder is defined from previous cells
# output_folder = '/content/gdrive/My Drive/devstats_report' # Uncomment if running this cell independently

html_content_parts = []

# Ensure analysis_run_date is available. It should be defined in a preceding cell.
# Add a safety check for analysis_run_date
if 'analysis_run_date' not in globals():
    analysis_run_date = "Analysis date not available - please run the 'Get repository info' cell." # Fallback
    print("Warning: 'analysis_run_date' not found. Using fallback message.")

print(f"Using analysis_run_date for report: {analysis_run_date}")

# 1. HTML Head and Styles
html_content_parts.append(f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Project Report: {repo.name}</title>
    <style>
        body {{
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f4f7f6;
            color: #333;
            line-height: 1.6;
        }}
        .container {{
            max-width: 1200px;
            margin: 20px auto;
            background-color: #ffffff;
            padding: 30px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }}
        h1, h2, h3, h4, h5, h6 {{
            color: #2c3e50;
        }}
        h1 {{
            text-align: center;
            color: #2196f3;
            margin-bottom: 30px;
        }}
        .section {{
            margin-bottom: 40px;
            border-bottom: 1px solid #eee;
            padding-bottom: 20px;
        }}
        .section:last-child {{
            border-bottom: none;
        }}
        .figure-container {{
            max-width: 1000px; /* Increased max-width for centering wider iframes */
            margin: 20px auto; /* Center the figure container itself */
            text-align: center; /* Center inline content like h3 */
            border: 1px solid #ddd;
            padding: 15px;
            border-radius: 5px;
            background-color: #f9f9f9;
        }}
        .figure-container h3 {{
            margin-top: 0;
            color: #555;
        }}
        .figure-container iframe {{
            /* Removed default width: 100%; to allow individual width or auto-sizing */
            height: 750px; /* Adjusted default height for plotly figures to prevent flattening */
            width: 100%; /* Allow iframe to fill its container */
            border: none;
            display: block; /* Treat iframe as a block element */
            margin: 0 auto; /* Center iframe horizontally */
            text-align: center; /* Center iframe text */
        }}

        .subsection {{
            margin-bottom: 20px;
            border-bottom: 1px solid #eee;
            padding-bottom: 15px;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>Project Report: {repo.name}</h1>
"""
)

# Calculate total contributors
total_contributors = num_core_devs + num_externals

# 2. Project Overview (moved before Project Summary)
html_content_parts.append(f"""
        <div class="section">
            <h2>Analysis Period</h2>
            <p>{overview_description}</p>
            <p><b>Acquisition Date:</b> {analysis_run_date}</p>
            <p><b>Interval length:</b> {selected_period_str}</p>
        </div>
"""
)

# 3. Project Summary (moved after Project Overview)
html_content_parts.append(f"""
        <div class="section">
            <h2>Project Summary</h2>
            <p>{summary_description}</p>
            <p><b>Repository Name:</b> {repo.name}</p>
            <p><b>Description:</b> {repo.description if repo.description else 'No description provided.'}</p>
            <p><b>URL:</b> <a href="{repo.html_url}">{repo.html_url}</a></p>
            <p><b>Stars:</b> {repo.stargazers_count}</p>
            <p><b>Forks:</b> {repo.forks_count}</p>
            <p><b>Created At:</b> {repo.created_at.strftime('%Y-%m-%d %H:%M:%S')}</p>
            <p><b>Last Updated At:</b> {repo.updated_at.strftime('%Y-%m-%d %H:%M:%S')}</p>
            <p><b>Number of contributors:</b> {total_contributors}</p>
            <ul>
                <li>Project maintainers: {num_core_devs}</li>
                <li>External contributors: {num_externals}</li>
            </ul>
        </div>
"""
)

# 5. All Primitive Events
html_content_parts.append(f"""
        <div class="section">
            <h2>Events</h2>
            <p>{primitive_events_description}</p>
            <div class="figure-container">
                <iframe src="./all_primitive_events.html"></iframe>
            </div>
        </div>
"""
)

# 6. All Aggregate Events
html_content_parts.append(f"""
        <div class="section">
            <h2>Categorized events</h2>
            <p>{aggregate_events_description}</p>
            <div class="figure-container">
                <iframe src="./all_aggregate_events.html"></iframe>
            </div>
        </div>
"""
)

# 7. WB
html_content_parts.append(f"""
        <div class="section">
            <h2>Workload Balance (WB)</h2>
            <p>{wbi_description}</p>
            <div class="figure-container">
                <iframe src="./workload_balance.html" style="height: 200px; width: 880px;"></iframe>
            </div>
        </div>
"""
)

# 8. RB
html_content_parts.append(f"""
        <div class="section">
            <h2>Resolution Balance (RB)</h2>
            <p>{rbi_description}</p>
            <div class="figure-container">
                <iframe src="./resolution_balance.html" style="height: 200px; width: 880px;"></iframe>
            </div>
        </div>
"""
)

# 9. Project Performance Map
html_content_parts.append(f"""
        <div class="section">
            <h2>Project tide chart (WB x RB)</h2>
            <p>{performance_map_description}</p>
            <div class="figure-container">
                <iframe src="./wb_rb_map.html" style="height: 700px; width: 960px;"></iframe>
            </div>
        </div>
"""
)

# 10. PRs Close Time section
html_content_parts.append(f"""
        <div class="section">
            <h2>PR completion time</h2>
            <p>{pr_close_time_description}</p>
            <div class="figure-container">
                <iframe src="./pr_close_times_histogram.html"></iframe>
            </div>
            <div class="figure-container">
                <iframe src="./pr_close_time_scatter_plot.html" style="height: 700px;"></iframe>
            </div>
            <div class="figure-container">
                <iframe src="./pr_categories_pie_chart.html"></iframe>
            </div>
        </div>
"""
)

# 11. Developers Overview - Engagement
html_content_parts.append(f"""
        <div class="section">
            <h2>Developers Overview</h2>
            <div class="subsection">
                <h3>Engagement chart</h3>
                <p>{engagement_plot_description}</p>
                <div class="figure-container">
                    <iframe src="./engagement_plot.html" style="height: 700px;"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Engagement Magnitude Index (EM)</h3>
                <p>{remi_index_description}</p>
                <div class="figure-container">
                    <iframe src="./em_index.html"></iframe>
                </div>
            </div>
"""
)

# 12. Developers Overview - Delivery
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Delivery chart</h3>
                <p>{delivery_plot_description}</p>
                <div class="figure-container">
                    <iframe src="./delivery_plot.html" style="height: 900px;"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Delivery Magnitude Index (DM)</h3>
                <p>{rdmi_index_description}</p>
                <div class="figure-container">
                    <iframe src="./dm_index.html"></iframe>
                </div>
            </div>
"""
)

# 13. Engagement-Delivery Map
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Contribution chart</h3>
                <p>{engagement_delivery_map_description}</p>
                <div class="figure-container">
                    <iframe src="./engagement_delivery_map.html" style="height: 1000px;"></iframe>
                </div>
            </div>
"""
)

# 14. Contribution (EM+DM) Plot
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Contribution rank</h3>
                <p>{contribution_plot_description}</p>
                <div class="figure-container">
                    <iframe src="./contribution_em_dm_plot.html" style="height: 700px;"></iframe>
                </div>
            </div>
"""
)

# 15. Score Plot
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Contribution score</h3>
                <p>{score_plot_description}</p>
                <div class="figure-container">
                    <iframe src="./score_plot.html"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Contribution score percentiles</h3>
                <p>{score_percentiles_description}</p>
                <div class="figure-container">
                    <iframe src="./score_percentiles_box_plot.html"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Contribution score histogram</h3>
                <p>{score_histogram_description}</p>
                <div class="figure-container">
                    <iframe src="./score_histogram.html"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Score ranges</h3>
                <p>{score_proportions_description}</p>
                <div class="figure-container">
                    <iframe src="./score_proportions_pie_chart.html"></iframe>
                </div>
            </div>
        </div> <!-- Close Developers Overview section -->
"""
)

# 16. Aggregate Indices Header
html_content_parts.append(f"""
        <div class="section">
            <h2>Indices distributions</h2>
"""
)

# 18. Quartile Distribution of Key Development Metrics
html_content_parts.append(f"""
            <div class="subsection">
                <p>{metric_quartiles_description}</p>
                <div class="figure-container">
                    <iframe src="./metric_quartiles_box_plot.html" style="height: 700px;"></iframe>
                </div>
            </div>
        </div> <!-- Close Aggregate Indices section -->
"""
)

# 19. Inactive Developers (Moved to end)
inactive_devs_html = f"""
        <div class="section">
            <h2>Inactive Developers</h2>
            <p>{inactive_devs_description}</p>
"""
if not inactive_devs_df.empty:
    inactive_devs_html += f"""
            <p>{len(inactive_devs_df)} {inactive_devs_found_text}</p>
            <ul>
"""
    for index, row in inactive_devs_df.iterrows():
        dev_name = row['name'] if row['name'] != 'N/A' else 'Name not available'
        inactive_devs_html += f"                <li>{row['github_login']} ({dev_name})</li>\n"
else:
    inactive_devs_html += f"            <p>{inactive_devs_none_text}</p>\n"
inactive_devs_html += "        </div>\n"
html_content_parts.append(inactive_devs_html)

# 20. Final closing tags
html_content_parts.append("""
    </div> <!-- Close container -->
</body>
</html>
"""
)

# Join all parts and write to file
final_html_content = "".join(html_content_parts)

html_file_path = os.path.join(output_folder, 'index.html')

with open(html_file_path, 'w') as f:
    f.write(final_html_content)

print(f"Finalized HTML report created at: {html_file_path}")

Using analysis_run_date for report: 2025-11-30 14:06:54
Finalized HTML report created at: /content/gdrive/My Drive/naja-2025-11-30/index.html
